In [ ]:
import os
import plotly.graph_objects as go
import pandas as pd
from termcolor import colored
import signal
import numpy as np
from tabulate import tabulate
import imgkit
from scipy.stats import gaussian_kde
from joypy import joyplot
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import font_manager
from PIL import Image
import matplotlib.patches as mpatches
import pytz
import matplotlib.colors as mcolors


from helpers import * 
from xatu import *

WHAT = "head"
DATA = "data2"
TMP = "tmp"
IMAGE = f"{DATA}/images"
SHOW = True
DAYS = 31
OVERWRITE = False
CREATE_BEST_LIST = True
DO_PERFORMERS = False

IMAGE_HEIGHT = 530
FONTSIZE = 19

GOOGLE_CREDENTIALS = "./config/google-creds.json"

FOLDERS_TO_BE_USED = ["correct_target"]

FONTFAMILY = "Ubuntu Mono"

for i in FOLDERS_TO_BE_USED:
    if not os.path.isdir(f"{DATA}/{i}"):
        print("creating folder ", f"{DATA}/{i}")
        os.mkdir(f"{DATA}/{i}")

if not os.path.isdir(f"{IMAGE}"):
    print("creating folder ", f"{IMAGE}")
    os.mkdir(f"{IMAGE}")
try:
    os.environ['GOOGLE_APPLICATION_CREDENTIALS']
except:
    print(f"setting google credentials as global variable...")
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = GOOGLE_CREDENTIALS
    
stop_requested = False

def signal_handler(sig, frame):
    global stop_requested
    stop_requested = True
    print("Graceful shutdown requested...")

signal.signal(signal.SIGINT, signal_handler)
signal.signal(signal.SIGTERM, signal_handler)


COLORS = [
    '#377eb8', '#ff7f00', '#4daf4a', '#984ea3', '#f781bf', '#a65628', '#e41a1c', 
    '#a6cee3', '#999999', '#fdbf6f', '#b2df8a', '#fb9a99', '#cab2d6',
    '#1f78b4', '#33a02c', '#ff7f7f'
]

COLORS2 = [
    '#377eb8', '#ff7f00', '#4daf4a', '#984ea3', '#f781bf', '#a65628', '#e41a1c', 
    '#a6cee3', '#999999', '#fdbf6f', '#b2df8a', '#fb9a99', '#cab2d6',
    '#1f78b4', '#33a02c', '#ff7f7f', 
    '#8dd3c7', '#ffffb3', '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69',
    '#fccde5', '#d9d9d9', '#bc80bd', '#ccebc5', '#ffed6f'
]

COLORS3 = [
    '#377eb8', '#ff7f00', '#4daf4a', '#984ea3', '#f781bf', '#a65628',
    '#e41a1c', '#a6cee3', '#999999', '#fdbf6f', '#b2df8a', '#fb9a99',
    '#cab2d6', '#1f78b4', '#33a02c', '#ff7f7f', '#8dd3c7', '#ffffb3',
    '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69', '#fccde5',
    '#d9d9d9', '#bc80bd', '#ccebc5', '#ffed6f',
    '#6a3d9a', '#ffcc00', '#b15928', '#1f78b4', '#e31a1c', '#33a02c',
    '#fb9a99', '#e6ab02', '#a6761d', '#666666'
]

def get_labels(overwrite=False):
    try:
        if overwrite:
            print("overwriting...")
            raise
        labels = pd.read_parquet(f"{DATA}/{TMP}/labels.parquet").drop_duplicates()
        print("labels read locally")
    except:
        print("getting labeling info from bq")
        labels = pandas_gbq.read_gbq("""
            SELECT DISTINCT validator_id, label FROM `ethereum-data-nero.ethdata.beaconchain_validators_db` 
        """)
        labels.to_parquet(f"{DATA}/{TMP}/labels.parquet", index=False)
    return labels

def get_clients(overwrite=False):
    try:
        if overwrite:
            True
        clients = pd.read_parquet(f"{DATA}/{TMP}/clients_final.parquet").drop_duplicates()
        print("get_clients read locally")
    except:
        print("getting client info from bq")
        clients = pandas_gbq.read_gbq("""
            SELECT distinct validator_id, cl_client, slot FROM `ethereum-data-nero.ethdata.beaconchain_pace` 
        """).drop_duplicates()
        clients = clients[clients["cl_client"] != "missed"]
        clients = clients[clients["cl_client"] != "Unknown"]
        clients = clients.set_index("validator_id").to_dict()["cl_client"]
        pd.DataFrame(list(zip(clients.keys(), clients.values())), columns=["validator_id", "cl_client"]).to_parquet(f"{DATA}/{TMP}/clients_final.parquet", index=False)
        return get_clients(overwrite=False)
    return clients

def get_clients_with_slots(overwrite=False):
    try:
        if overwrite:
            True
        clients = pd.read_parquet(f"{DATA}/{TMP}/clients_final_with_slot.parquet").drop_duplicates()
        print("get_clients read locally")
    except:
        print("getting client info from bq")
        clients = pandas_gbq.read_gbq("""
            SELECT distinct validator_id, cl_client, slot FROM `ethereum-data-nero.ethdata.beaconchain_pace` 
        """).drop_duplicates()
        clients = clients[clients["cl_client"] != "missed"]
        clients = clients[clients["cl_client"] != "Unknown"]
        clients.drop_duplicates().to_parquet(f"{DATA}/{TMP}/clients_final_with_slot.parquet", index=False)
        return get_clients(overwrite=False)
    return clients


def get_block_sizes(df, overwrite=False):
    try:
        if overwrite:
            raise
        size = pd.read_parquet(f"{DATA}/{TMP}/sizes.parquet").drop_duplicates()
        print("get_block_sizes read locally")
    except:
        print("getting block size info from bq")
        size = pandas_gbq.read_gbq(f"""
            SELECT distinct slot, size, size_compressed, 128*1024*nr_blobs as blobs
            FROM `ethereum-data-nero.ethdata.ethereum_blocksize_pace` 
            where slot >= {df.slot.min()-5} and slot <= {df.slot.max()+5}
        """)
        size.to_parquet(f"{DATA}/{TMP}/sizes.parquet", index=False)
    return size

def read_df():
    print(f"reading {DATA}/failed_missed_{WHAT}/failed_missed_data.parquet")
    df = pd.read_parquet(f"{DATA}/failed_missed_{WHAT}/failed_missed_data.parquet").drop_duplicates()
    return df

def add_slot_in_epoch(df):
    df["slot_in_epoch"] = df["slot"] % 32
    return df

def merge_labels(df, labels, left_on="validator", right_on="validator_id", nlargest=15):
    df = pd.merge(df, labels, how="left", left_on=left_on, right_on=right_on).drop(right_on, axis=1)
    df["label"] = df["label"].apply(lambda x: "unidentified" if x == None else x)
    df["label"] = df["label"].fillna("unidentified")
    df["label"] = df["label"].apply(lambda x: "solo staker" if x.endswith(".eth") else x.lower())
    largest = df.groupby("label")["validator"].sum().reset_index().sort_values("validator", ascending=False)["label"].tolist()[0:nlargest]
    if "solo staker" not in largest:
        largest.append("solo staker")
    df["label"] = df["label"].apply(lambda x: x if x in largest else "other")
    if "other" not in largest:
        largest.append("other")
    return df, largest

def add_info_to_df(df):
    df = add_slot_in_epoch(df)
    #df.drop(["failed", "missed"], axis=1, inplace=True)
    #df.drop_duplicates(inplace=True)
    df, largest = merge_labels(df, labels)
    return df, largest

def check_duties(epoch):
    epoch_start = epoch * 32
    dfs = []
    for i in range(epoch_start, epoch_start + 32):
        df = pd.read_parquet(f"{DATA}/duties/{i}.parquet").drop_duplicates()
        df['slot'] = i
        dfs.append(df)
    return pd.concat(dfs).reset_index(drop=True)

def handle_labels(_df, nlargest=15):
    df = _df.copy()
    df["label"] = df["label"].apply(lambda x: "unidentified" if x == None else x)
    df["label"] = df["label"].fillna("unidentified")
    df["label"] = df["label"].apply(lambda x: "solo staker" if x.endswith(".eth") else x.lower())
    largest = df.groupby("label")["validator"].sum().reset_index().sort_values("validator", ascending=False)["label"].tolist()[0:nlargest]
    if  "solo staker" not in largest:
        largest.append("solo staker")
    df["label"] = df["label"].apply(lambda x: x if x in largest else "other")
    return df


def get_totals(epoch, nlargest=15):
    duties = check_duties(epoch)
    duties = pd.merge(duties, labels, how="left", left_on="validator", right_on="validator_id").drop("validator_id", axis=1)
    duties = handle_labels(duties, nlargest)
    totals = duties.groupby("label")["validator"].nunique().reset_index().sort_values("validator")
    return totals

def get_all_misses_and_totals(df, nlargest=15):
    misses = df.groupby("label")["validator"].nunique().reset_index().sort_values("validator")
    totals = get_totals(df.epoch.max())
    return totals, misses

def get_offline_validator_count(df):
    print("offline validators")
    missed = df.groupby(["validator", "label"])["slot"].nunique().reset_index().sort_values("slot")
    mi = missed[missed["slot"] > 1].reset_index(drop=True)
    print(f"offline for {mi.slot.sum()} slots, {mi.slot.sum() // 32} epochs")
    mi = mi[mi["slot"] == mi.slot.max()].groupby("label")["validator"].nunique().reset_index().sort_values("validator")
    print(mi.to_markdown())

def add_date_to_df(df):
    BASE_TIMESTAMP = 1606824023
    SLOT_DURATION = 12
    timestamps = BASE_TIMESTAMP + pd.Series(df["slot"]) * SLOT_DURATION
    dt_objects = pd.to_datetime(timestamps, unit='s')
    formatted_times = dt_objects.dt.strftime("%Y-%m-%d %H:00") 
    df["date"] = formatted_times
    return df

def add_day_to_df(df):
    BASE_TIMESTAMP = 1606824023
    SLOT_DURATION = 12
    timestamps = BASE_TIMESTAMP + pd.Series(df["slot"]) * SLOT_DURATION
    dt_objects = pd.to_datetime(timestamps, unit='s')
    formatted_times = dt_objects.dt.strftime("%Y-%m-%d") 
    df["date"] = formatted_times
    return df

def combine_failed_and_missed_columns(df):
    conditions = [
        (df['failed'] == 1),
        (df['missed'] == 1)
    ]

    choices = ['failed', 'missed']
    df['status'] = np.select(conditions, choices, default='success')
    #df.drop(["missed", "failed"], axis=1, inplace=True)
    df.replace("failed", f"wrong {WHAT} vote", inplace=True)
    df.replace("missed", f"missed/offline", inplace=True)
    return df

def get_slot_per_epoch_totals_dff(gdf_grouped):
    dff = dict()
    for i in gdf_grouped.slot_in_epoch.unique():
        _dff = gdf_grouped[gdf_grouped["slot_in_epoch"] == i]
        dff[i] = _dff.validator.sum()
    return dff

def get_time_in_curr_slot(ts, slot):
    return (ts - (1654824023000 + (slot-1-4e6)*12000) - 12000) / 1000 

def load_timing_data(df, overwrite=False, rounding=1):
    try:
        if overwrite == True:
            raise
        block_timing = pd.read_parquet(f"{DATA}/{TMP}/block_timings.parquet")
        block_timing["time_in_slot"] = block_timing.apply(lambda x: get_time_in_curr_slot(x["timestamp_ms"], x["slot"]), axis=1)
        block_timing.dropna(subset="time_in_slot", inplace=True)
        block_timing["time_in_slot"] = block_timing["time_in_slot"].astype(float)
        block_timing["time_in_slot"] = block_timing["time_in_slot"].round(rounding)     
        print("timing_data read locally")
        return block_timing[["slot", "time_in_slot"]]
    except Exception as e:
        print(e)
        block_timing = pandas_gbq.read_gbq(f"""
            SELECT distinct A.slot, A.block_hash, min(timestamp_ms) timestamp_ms , 1 as mevboost
              FROM (
                      SELECT slot, block_hash 
                      FROM `ethereum-data-nero.eth.mevboost_db` 
                      WHERE date > "2024-05-25"
                  ) A LEFT JOIN (
                      SELECT block_hash, slot, timestamp_ms 
                      FROM `ethereum-data-nero.eth.mevboost_all_bids` 
                      UNION ALL
                      SELECT block_hash, slot, timestamp_ms fROM `ethereum-data-nero.eth.mevboost_all_bids_archive_0`
                      UNION ALL
                      SELECT block_hash, slot, timestamp_ms fROM `ethereum-data-nero.eth.mevboost_all_bids_archive_1`
                      UNION ALL
                      SELECT block_hash, slot, timestamp_ms fROM `ethereum-data-nero.eth.mevboost_all_bids_archive_4`

          ) B on A.slot = B.slot and A.block_hash = B.block_hash
          where A.slot >= {df.slot.min()}
          group by slot, block_hash
        """)
        block_timing["time_in_slot"] = block_timing.apply(lambda x: get_time_in_curr_slot(x["timestamp_ms"], x["slot"]), axis=1)
        block_timing.dropna(subset="time_in_slot", inplace=True)
        block_timing["time_in_slot"] = block_timing["time_in_slot"].astype(float)
        block_timing["time_in_slot"] = block_timing["time_in_slot"].round(rounding)
        block_timing.to_parquet(f"{DATA}/{TMP}/block_timings.parquet", index=False)        
        return block_timing[["slot", "time_in_slot"]]
    
def prepare_failed_with_size(_df, only_failed=True, overwrite=False):
    if only_failed:
        df = _df[_df["failed"] == 1].copy()
    else:
        df = _df.copy()
    size = get_block_sizes(df, overwrite)
    size["size_compressed_total"] = size["size_compressed"] + size["blobs"]   
    df = pd.merge(df, size[["slot", "size", "size_compressed", "size_compressed_total", "blobs"]], how="left", left_on="slot", right_on="slot")
    aggr = {
        "size": "mean",
        "size_compressed": "mean",
        "validator": "nunique",
        "size_compressed_total": "mean",
        "blobs": "mean"
    }
    df = df.groupby("slot")[["size", "size_compressed", "validator", "size_compressed_total", "blobs"]].agg(aggr).reset_index()
    return df

def remove_last_day(_df):
    df = _df.copy()
    df = add_day_to_df(df)
    return df[df["date"] != df.date.max()]

def get_share(ee):
    return ee.groupby("time_in_slot")["slot"].nunique().reset_index().sort_values("slot", ascending=False).set_index("time_in_slot").to_dict()["slot"]

In [ ]:
df = read_df()
print(len(df))
df = remove_last_day(df)
print(len(df))
df = df[df["epoch"] > df.epoch.max() - DAYS*225]
print(len(df))
print(f"having {df.epoch.nunique()} epochs, {df.epoch.nunique()//225} days")

labels = get_labels(overwrite=OVERWRITE)

clients_final = get_clients(overwrite=OVERWRITE)

df, largest = add_info_to_df(df)

get_offline_validator_count(df)

df = add_date_to_df(df)

block_timing = load_timing_data(df, overwrite=OVERWRITE)

totals, misses = get_all_misses_and_totals(df, nlargest=15)

In [ ]:
#gaps
print("gaps: ", str(set(range(df.epoch.min(), df.epoch.max()+1)) - set(df.epoch)))

In [ ]:
print("first/last slot ind data: ", f"{df.slot.min()}/{df.slot.max()}")
print("first/last slot ind data: ", f"{slot_to_hour(df.slot.min())}/{slot_to_hour(df.slot.max())}")

In [ ]:
legit = set(check_duties(df.epoch.min())["validator"].tolist()).intersection(set(check_duties(df.epoch.max())["validator"].tolist()))
len(legit)

In [ ]:
_df =  df[df["epoch"] > df.epoch.max() - 3*225].copy()
_df = _df[_df.validator.isin(legit)]
_df = _df.groupby("validator")["epoch"].nunique().reset_index()
_df

In [ ]:
totals, misses = get_all_misses_and_totals(df, nlargest=30)

In [ ]:
# no misses
m = set(_df["validator"])
m = [i for i in legit if i not in m]
len(m), len(m)-len(legit)

In [ ]:
no_misses = labels[labels["validator_id"].isin(m)]
no_misses.columns = ["validator", "label"]

In [ ]:
no_misses = handle_labels(no_misses, 30)
no_misses

In [ ]:
no_misses.label.unique()

In [ ]:
no_misses = no_misses.groupby("label")["validator"].nunique().reset_index().sort_values("validator")
no_misses["per"] = no_misses["validator"]/no_misses.validator.sum()*100
no_misses["label"] = no_misses["label"].str.lower()

In [ ]:
no_misses

In [ ]:
totals["per"] = totals["validator"]/totals.validator.sum()*100
totals

In [ ]:
no_miss_total = pd.merge(totals[["label", "per"]], no_misses[["label", "per"]], how="inner", left_on="label", right_on="label")


In [ ]:
no_miss_total.sort_values("per_x", ascending=False, inplace=True)
no_miss_total

### Worst validators

In [ ]:
def head_votes_over_validator_ids(df):
    _df = df[df.validator.isin(legit)].copy()
    _df = _df.groupby("validator")["epoch"].nunique().reset_index()
    _df["valr"] = _df["validator"] // 10000 * 10000
    dd = _df.groupby("valr")["epoch"].sum().reset_index()
    dd["epoch"] = dd["epoch"] / df.epoch.nunique()
    fig = go.Figure()
    fig.add_trace(go.Bar(
            x=dd['valr'],
            y=dd['epoch'],
            name='Share in Top Performing Validators (%)',
            marker_color=COLORS[0]
        ))

    fig.update_layout(
        title=f'Missed/Failed {WHAT[0].upper() + WHAT[1:]} Votes Over Validator IDs <span style="font-size: 16px;">({slot_to_day(df.slot.min())} - {slot_to_day(df.slot.max())})</span>',
        xaxis_title="validator id",
        yaxis_title=f'missed/failed {WHAT} votes',
        barmode='group',
        plot_bgcolor='white',
        font=dict(family="Ubuntu Mono", size=FONTSIZE),
        legend=dict(
            x=1,
            y=1,
            xanchor='right',
            yanchor='top'
        ),
        xaxis=dict(showgrid=True, gridcolor='lightgray', tickformat=",d"),
        yaxis=dict(showgrid=True, gridcolor='lightgray',),
        height=550,
        width=1200,
        )
    return fig

fig = head_votes_over_validator_ids(df)
fig.write_image(f"{IMAGE}/{WHAT}_votes_over_validator_ids.png")
if SHOW:
    fig.show()

In [ ]:
def head_votes_over_offline_validator_ids(df):
    _df = df[df.validator.isin(legit)].copy()
    _df = _df.groupby("validator")["epoch"].nunique().reset_index()
    offline = _df[_df["epoch"] == _df.epoch.max()]
    offline["valr"] = offline["validator"] // 10000 * 10000
    dd = offline.groupby("valr")["validator"].count().reset_index()
    #dd["epoch"] = dd["epoch"] / df.epoch.nunique()
    fig = go.Figure()
    fig.add_trace(go.Bar(
            x=dd['valr'],
            y=dd['validator'],
            name='Share in Top Performing Validators (%)',
            marker_color=COLORS[0]
        ))

    fig.update_layout(
        title=f'Offline Validators Over Validator IDs <span style="font-size: 16px;">({slot_to_day(df.slot.min())} - {slot_to_day(df.slot.max())})</span>',
        xaxis_title="Validator ID",
        yaxis_title=f'missed/failed {WHAT} votes',
        barmode='group',
        plot_bgcolor='white',
        font=dict(family="Ubuntu Mono", size=FONTSIZE),
        legend=dict(
            x=1,
            y=1,
            xanchor='right',
            yanchor='top'
        ),
        xaxis=dict(showgrid=True, gridcolor='lightgray', tickformat=",d"),
        yaxis=dict(showgrid=True, gridcolor='lightgray',),
        height=550,
        width=1200,
        )
    return fig

fig = head_votes_over_offline_validator_ids(df)
fig.write_image(f"{IMAGE}/{WHAT}_votes_over_offline_validator_ids.png")
if SHOW:
    fig.show()

In [ ]:
def head_votes_over_bad_validator_ids(df):
    _df = df[df.validator.isin(legit)].copy()
    _df = _df.groupby("validator")["epoch"].nunique().reset_index()
    noffline = _df[_df["epoch"] != _df.epoch.max()]
    noffline = noffline[noffline["epoch"] > noffline.epoch.mean()]
    noffline["valr"] = noffline["validator"] // 10000 * 10000
    dd = noffline.groupby("valr")["validator"].count().reset_index()
    #dd["epoch"] = dd["epoch"] / df.epoch.nunique()
    fig = go.Figure()
    fig.add_trace(go.Bar(
            x=dd['valr'],
            y=dd['validator'],
            name='Share in Top Performing Validators (%)',
            marker_color=COLORS[0]
        ))

    fig.update_layout(
        title=f'Bad Validators Over Validator IDs <span style="font-size: 16px;">({slot_to_day(df.slot.min())} - {slot_to_day(df.slot.max())})</span>',
        xaxis_title="validator id",
        yaxis_title=f'missed/failed {WHAT} votes',
        barmode='group',
        plot_bgcolor='white',
        font=dict(family="Ubuntu Mono", size=FONTSIZE),
        legend=dict(
            x=1,
            y=1,
            xanchor='right',
            yanchor='top'
        ),
        xaxis=dict(showgrid=True, gridcolor='lightgray', tickformat=",d"),
        yaxis=dict(showgrid=True, gridcolor='lightgray',),
        height=550,
        width=1200,
        )
    fig.add_annotation(
            text="* bad validators miss more than avg. but are not offline",
            xref="paper",
            yref="paper",
            x=0.95,
            y=1,
            showarrow=False,
            font=dict(
                family=FONTFAMILY,
                size=FONTSIZE-2,
                color="black"
            )
        )
    return fig

fig = head_votes_over_bad_validator_ids(df)
fig.write_image(f"{IMAGE}/{WHAT}_votes_over_bad_validator_ids.png")
if SHOW:
    fig.show()

In [ ]:
def performance_votes_best(df):
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=df['label'],
        y=df['per_x'],
        name='Total Share (%)',
        marker_color=COLORS[0]
    ))

    fig.add_trace(go.Bar(
        x=df['label'],
        y=df['per_y'],
        name='Share in Top Performing Validators (%)',
        marker_color=COLORS[1]
    ))

    fig.update_layout(
        title='Total Validator Share vs. Share in Top Performing Validators',
        xaxis_title=None,
        yaxis_title='%',
        barmode='group',
        plot_bgcolor='white',
        font=dict(family="Ubuntu Mono", size=FONTSIZE),
        legend=dict(
            x=1,
            y=1,
            xanchor='right',
            yanchor='top'
        ),
        xaxis=dict(showgrid=True, gridcolor='lightgray'),
        yaxis=dict(showgrid=True, gridcolor='lightgray', range=[0, 100])
    )
    return fig
#
#fig = performance_votes_best(no_miss_total)
#fig.write_image(f"{IMAGE}/performer_vs_total.png")
#if SHOW:
#    fig.show()

In [ ]:
#no_miss_total
#no_miss_total2 = pd.merge(totals[["label", "validator"]], no_misses[["label", "validator"]], how="inner", left_on="label", right_on="label")
#no_miss_total2["per"] = no_miss_total2["validator_y"] / no_miss_total2["validator_x"] * 100
#no_miss_total2.sort_values("per", ascending=False, inplace=True)
#no_miss_total2

def performance_votes_best2(df):
    start_color = '#377eb8'
    end_color = '#8cb6fa'
    cmap = mcolors.LinearSegmentedColormap.from_list("custom_gradient", [start_color, end_color])
    gradient = [mcolors.rgb2hex(cmap(i / 19)) for i in range(20)]
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=df['label'],
        y=df['per'],
        name='Total Share (%)',
        marker_color=gradient
    ))

    fig.update_layout(
        title='Total Validator Share vs. Share in Top Performing Validators',
        xaxis_title=None,
        yaxis_title='%',
        barmode='group',
        plot_bgcolor='white',
        font=dict(family="Ubuntu Mono", size=FONTSIZE),
        legend=dict(
            x=1,
            y=1,
            xanchor='right',
            yanchor='top'
        ),
        xaxis=dict(showgrid=True, gridcolor='lightgray'),
        yaxis=dict(showgrid=True, gridcolor='lightgray',)
    )
    return fig

#fig = performance_votes_best2(no_miss_total2)
#fig.write_image(f"{IMAGE}/performer_share_{WHAT}.png")
#if SHOW:
#    fig.show()

In [ ]:
files = os.listdir(f"{DATA}/failed_{WHAT}/")

duties_files = os.listdir(f"{DATA}/duties/")
duties_files = set([f"{int(i.split('.')[0])//32}.parquet" for i in duties_files])
files = sorted([f  for f in files if f in duties_files], key=lambda x: int(x.split(".")[0]))
print(f"slots in files: {len(files)*32} | {len(files)*32//7200} days of data")
print(f"First file:\n{files[0]}")
print(f"Last file:\n{files[-1]}")
print(f'{labels.dropna()[labels.dropna()["label"].str.endswith(".eth")].validator_id.nunique()} unique solo stakers')

In [ ]:
def missed_votes_over_slot_in_eooch(_df):
    df = _df.copy()
    df.drop(["failed", "missed"], axis=1, inplace=True)
    df.drop_duplicates(inplace=True)
    df = df.groupby(["slot_in_epoch", "label"])["validator"].count().reset_index().sort_values("slot_in_epoch")
    df = df.reset_index(drop=True)
    df["validator"] = df["validator"] / (_df.slot.nunique()/32)

    fig = go.Figure()

    for ix, label in enumerate(largest):
        df_label = df[df['label'] == label]
        fig.add_trace(go.Bar(
            x=df_label['slot_in_epoch'],
            y=df_label['validator'],
            name=label,
            marker_color=COLORS2[ix]
        ))

    fig.update_layout(
        title=f'Missed/Failed {WHAT[0].upper() + WHAT[1:]} Votes over Slot In Epoch <span style="font-size: 16px;">({slot_to_day(_df.slot.min())} - {slot_to_day(_df.slot.max())})</span>',
        barmode='stack',
        legend_traceorder="normal",
        xaxis=dict(
            tickmode='linear',
            showgrid=True,
            gridcolor='lightgrey',
            title="slot in epoch"
        ),
        yaxis=dict(
            title=f"missed/failed {WHAT} votes per slot",
            showgrid=True,
            gridcolor='lightgrey',
        ),
        height=IMAGE_HEIGHT*1.25,
        width=1200,
        font=dict(
            family="Ubuntu Mono",
            size=FONTSIZE,
            color="black"
        ),
        plot_bgcolor = "#FFFFFF"
        #yaxis = dict(
        #    type="log"
        #)
    )
    return fig

fig = missed_votes_over_slot_in_eooch(df)
fig.write_image(f"{IMAGE}/missed_{WHAT}_votes_over_slot.png")
if SHOW:
    fig.show()
print("done")

In [ ]:
#dfm = pd.merge(misses, totals, how="left", left_on="label", right_on="label")
#dfm["validator_x"] = dfm["validator_x"] / dfm["validator_x"].sum() *100
#dfm["validator_y"] = dfm["validator_y"] / dfm["validator_y"].sum() *100
#dfm = dfm.iloc[::-1]

#fig = go.Figure()
#
#colors = [
#    '#377eb8', '#ff7f00', '#4daf4a', '#984ea3', '#f781bf', '#a65628', '#e41a1c', 
#    '#a6cee3', '#999999', '#fdbf6f', '#b2df8a', '#fb9a99', '#cab2d6',
#    '#1f78b4', '#33a02c', '#ff7f7f'
#]
#
## Create subplots: use 'domain' type for pie-like plots
#fig = make_subplots(specs=[[{"secondary_y": False}]])
#
## Add line trace for expected misses
#fig.add_trace(
#    go.Scatter(x=dfm['label'], y=dfm['validator_y'], name='Validator Marketshare', mode='lines+markers', 
#               line=dict(color='darkblue'), marker=dict(color='darkblue')),
#)
#
#
## Add bar trace for actual misses
#fig.add_trace(
#    go.Bar(x=dfm['label'], y=dfm['validator_x'], name='Missed Source Marketshare',
#    marker_color=colors[:len(dfm['label'])]
#          )
#)
#
#fig.update_layout(
#    title_text=f'Comparison of Expected and Actual Misses per Validator <span style="font-size: 14px;">({slot_to_day(df.slot.min())} - {slot_to_day(df.slot.max())})</span>',
#    legend=dict(x=0.9, y=0.98),
#    barmode='group',
#    legend_traceorder="normal",
#    xaxis=dict(
#        tickmode='linear',
#        showgrid=True,
#        gridcolor='lightgrey',
#        title=None
#    ),
#    yaxis=dict(
#        title="%",
#        showgrid=True,
#        gridcolor='lightgrey',
#        #type="log"
#    ),
#    height=550,
#    width=1200,
#    font=dict(
#        family="Ubuntu Mono",
#        size=14,
#        color="black"
#    ),
#    plot_bgcolor = "#FFFFFF"
#)
#
## Show plot
#fig.show()

In [ ]:
#dfm["delta"] = dfm["validator_x"] - dfm["validator_y"]
##dfm["delta_per"] = dfm["delta"] / dfm["validator_x"] * 100
#dfm.columns = ["Validator", "Share in missed source votes (%)", "Share in total validator market (%)", "Delta"]
#
#print(dfm.iloc[::-1].to_markdown())

In [ ]:
def create_delta_chart(_df):
    df = _df.copy()
    df = df.drop(["failed", "missed"], axis=1)
    df.drop_duplicates(inplace=True)
    missed = df.groupby(["validator", "label"])["slot"].nunique().reset_index().sort_values("slot")
    mi = missed[missed["slot"] > 1].reset_index(drop=True)
    mis = pd.merge(missed.groupby("label")["slot"].sum().reset_index().sort_values("slot"), totals, how="left", left_on = "label", right_on = "label")
    mis["validator"] = mis["validator"] * df.epoch.nunique()
    mis["actual_percentage"] = mis["slot"] / mis["validator"]*100
    mis.sort_values("actual_percentage", ascending=False).reset_index(drop=True)
    mis["expected_percentage"] = (mis["slot"].sum()/mis["validator"].sum())*100
    mis["greater"] = mis["actual_percentage"] > mis["expected_percentage"]
    mis = mis[["label", "actual_percentage", "expected_percentage", "greater"]]
    mis["delta"] = mis["actual_percentage"] - mis["expected_percentage"]
    colors = mis["delta"].apply(lambda x: COLORS[0] if x > 0 else COLORS[1])
    mis.sort_values("delta", inplace=True)    
    mis.dropna(subset="delta", inplace=True)
    fig = go.Figure()

    # Adding the bars
    fig.add_trace(go.Bar(
        x=mis["label"],
        y=mis["delta"],
        marker_color=mis["delta"].apply(lambda x: COLORS[0] if x > 0 else COLORS[1]),
        name='Performance Delta comparing `Actual` with `Expected`'
    ))

    # Adding the expected percentage line
    fig.add_trace(go.Scatter(
        x=mis["label"],
        y=[0] * len(mis),
        mode='lines',
        line=dict(color='black', dash='dash', width=3),
        name='Expected Missed Source Votes based on Market Share'
    ))

    # Updating layout
    fig.update_layout(
        title_text=f'Comparison of Expected and Actual Misses per Validator <span style="font-size: 16px;">({slot_to_day(df.slot.min())} - {slot_to_day(df.slot.max())})</span>',
        legend=dict(x=0, y=1),
        barmode='group',
        legend_traceorder="reversed",
        xaxis=dict(
            tickmode='linear',
            showgrid=True,
            gridcolor='lightgrey',
            title=None
        ),
        yaxis=dict(
            title="%",
            showgrid=True,
            gridcolor='lightgrey',
            #type="log"
             range=[-3, 3.5]
        ),
        height=IMAGE_HEIGHT*1.1,
        width=1200,
        font=dict(
            family="Ubuntu Mono",
            size=FONTSIZE,
            color="black"
        ),
        plot_bgcolor = "#FFFFFF"

    )
    return fig

fig = create_delta_chart(df)
fig.write_image(f"{IMAGE}/delta_missed_{WHAT}_votes.png")
if SHOW:
    fig.show()
print("done")

In [ ]:
def missed_votes_over_time(_df):
    df = _df.copy()
    df = df.drop(["failed", "missed"], axis=1)
    df.drop_duplicates(inplace=True)
    df["date"] = df["date"].str.split(" ").str[0]
    df = df.groupby(["date", "label"])["validator"].count().reset_index()
    df["validator"] = df["validator"] / (60*60*24/12)    
    fig = go.Figure()

    # Add a bar for each label
    for ix, label in enumerate(largest):
        df_label = df[df['label'] == label]
        fig.add_trace(go.Bar(
            x=df_label['date'],
            y=df_label['validator'],
            name=label,
            marker_color=COLORS2[ix % len(COLORS2)]
            
        ))

    fig.update_layout(
        title=f'Average Number of Missed/Failed {WHAT[0].upper() + WHAT[1:]} Votes per Slot',
        barmode='stack',
        legend_traceorder="normal",
        xaxis=dict(
            tickmode='linear',
            showgrid=True,
            gridcolor='lightgrey',
            title=None
        ),
        yaxis=dict(
            title=f"missed/failed {WHAT} votes per slot",
            showgrid=True,
            gridcolor='lightgrey',
        ),
        height=IMAGE_HEIGHT*1.25,
        width=1200,
        font=dict(
            family="Ubuntu Mono",
            size=FONTSIZE,
            color="black"
        ),
        plot_bgcolor = "#FFFFFF"
        #yaxis = dict(
        #    type="log"
        #)
    )
    return fig

fig = missed_votes_over_time(df)
fig.write_image(f"{IMAGE}/missed_{WHAT}_votes_over_date.png")
if SHOW:
    fig.show()
print("done")

In [ ]:
def missed_reorged_over_slots(_df):
    df = _df.copy()
    df = combine_failed_and_missed_columns(df)
    df = df.groupby(["slot_in_epoch", "status"])["validator"].count().reset_index().sort_values("slot_in_epoch")
    df = df.drop_duplicates().reset_index(drop=True)
    df.replace("failed", f"wrong {WHAT} vote", inplace=True)
    df.replace("missed", f"missed/offline", inplace=True)
    df["validator"] = df["validator"] / (_df.slot.nunique()/32)

    fig = go.Figure()

    for ix, status in enumerate(df.status.unique()):
        df_status = df[df['status'] == status]
        fig.add_trace(go.Bar(
            x=df_status['slot_in_epoch'],
            y=df_status['validator'],
            name=status,
            marker_color=COLORS[ix]
        ))

    fig.update_layout(
        title=f'Missed/Failed {WHAT[0].upper() + WHAT[1:]} Votes over Slot In Epoch <span style="font-size: 16px;">({slot_to_day(_df.slot.min())} - {slot_to_day(_df.slot.max())})</span>',
        barmode='stack',
        legend_traceorder="normal",
        legend=dict(x=0.81, y=1),
        xaxis=dict(
            tickmode='linear',
            showgrid=True,
            gridcolor='lightgrey',
            title="slot in epoch"
        ),
        yaxis=dict(
            title=f"missed/failed {WHAT} votes",
            showgrid=True,
            gridcolor='lightgrey',
        ),
        height=550,
        width=1200,
        font=dict(
            family="Ubuntu Mono",
            size=FONTSIZE,
            color="black"
        ),
        plot_bgcolor = "#FFFFFF"
        #yaxis = dict(
        #    type="log"
        #)
    )
    return fig

fig = missed_reorged_over_slots(df)
fig.write_image(f"{IMAGE}/failed_missed_{WHAT}_votes.png")
if SHOW:
    fig.show()
print("done")

In [ ]:
def failed_misse_percentage_over_slot(_df):
    df = _df.copy()
    min_max_slots = (df.slot.min(), df.slot.max())
    dff = get_slot_per_epoch_totals_dff(
        combine_failed_and_missed_columns(df).groupby(
            ["slot_in_epoch", "status"]
        )["validator"].count().reset_index().sort_values("slot_in_epoch")
    ) 
    df = df.groupby(["slot_in_epoch", "status"])["validator"].count().reset_index().sort_values("slot_in_epoch")
    df["validator"] = df.apply(lambda x: x["validator"]/dff[x["slot_in_epoch"]] * 100, axis=1)
    
    fig = go.Figure()
    colors = [COLORS[1]] + [COLORS[0]]
    for ix, status in enumerate(["missed/offline", f"wrong {WHAT} vote"][::-1]):
        df_status = df[df['status'] == status]
        fig.add_trace(go.Bar(
            x=df_status['slot_in_epoch'],
            y=df_status['validator'],
            name=status,
            marker_color=colors[ix]
        ))

    fig.update_layout(
        title=f'Missed/Failed {WHAT[0].upper() + WHAT[1:]} Votes over Slot In Epoch <span style="font-size: 16px;">({slot_to_day(min_max_slots[0])} - {slot_to_day(min_max_slots[1])})</span>',
        barmode='stack',
        legend=dict(x=0.79, y=0.02),
        legend_traceorder="normal",
        xaxis=dict(
            tickmode='linear',
            showgrid=True,
            gridcolor='lightgrey',
            title="slot in epoch"
        ),
        yaxis=dict(
            title="%",
            showgrid=True,
            gridcolor='lightgrey',
        ),
        height=550,
        width=1200,
        font=dict(
            family="Ubuntu Mono",
            size=FONTSIZE,
            color="black"
        ),
        plot_bgcolor = "#FFFFFF"
        #yaxis = dict(
        #    type="log"
        #)
    )
    return fig

fig = failed_misse_percentage_over_slot(df)
fig.write_image(f"{IMAGE}/failed_missed_{WHAT}_votes_per.png")
if SHOW:
    fig.show()
print("done")

In [ ]:
def missed_over_clients_over_time(_df, clients_final):
    df = _df.copy()
    df = pd.merge(df, clients_final, how="left", left_on="validator", right_on="validator_id")
    
    min_max_slots = (df.slot.min(), df.slot.max())
    df["cl_client"] = df["cl_client"].fillna("Unknown")
    BASE_TIMESTAMP = 1606824023
    SLOT_DURATION = 12
    timestamps = BASE_TIMESTAMP + pd.Series(df["slot"]) * SLOT_DURATION
    dt_objects = pd.to_datetime(timestamps, unit='s')
    df["hour"] = dt_objects.dt.strftime("%Y-%m-%d")
    #df["date"] = df["date"].str.split(" ").str[0]
    df = df.groupby(["cl_client", "hour"])["validator"].count().reset_index().sort_values("validator", ascending=False)
    df["validator"] = df["validator"] / (60*60*24/12)   
    #df.set_index("cl_client", inplace=True)
    #df = df.loc[["Lighthouse",  'Prysm', 'Teku', 'Nimbus', 'Lodestar', "Unknown"]]
    #df.reset_index(inplace=True)
    fig = go.Figure()

    # Add a bar for each label
    for ix, cl_client in enumerate(["Lighthouse",  'Prysm', 'Teku', 'Nimbus', 'Lodestar', "Unknown"]):
        df_cl_client = df[df['cl_client'] == cl_client]
        fig.add_trace(go.Bar(
            x=df_cl_client['hour'],
            y=df_cl_client['validator'],
            name=cl_client,
            marker_color=COLORS2[ix % len(COLORS2)]
        ))


    fig.update_layout(
        title=f'Missed/Failed {WHAT[0].upper() + WHAT[1:]} Votes per CL Client <span style="font-size: 16px;">({slot_to_day(min_max_slots[0])} - {slot_to_day(min_max_slots[1])})</span>',
        barmode='stack',
        legend_traceorder="normal",
        xaxis=dict(
            tickmode='linear',
            showgrid=True,
            gridcolor='lightgrey',
            title=None
        ),
        yaxis=dict(
            title=f"missed/failed {WHAT} votes per slot",
            showgrid=True,
            gridcolor='lightgrey',
        ),
        height=500,
        width=1200,
        font=dict(
            family="Ubuntu Mono",
            size=FONTSIZE,
            color="black"
        ),
        plot_bgcolor = "#FFFFFF"
        #yaxis = dict(
        #    type="log"
        #)
    )
    return fig

fig = missed_over_clients_over_time(df, clients_final)
fig.write_image(f"{IMAGE}/failed_missed_{WHAT}_votes_over_clclient.png")
if SHOW:
    fig.show()
print("done")

In [ ]:
def missed_over_clients_over_slots_in_epoch(df, clients_final):
    min_max_slots = (df.slot.min(), df.slot.max())
    ee = pd.merge(df, clients_final, how="left", left_on="validator", right_on="validator_id")
    ee["cl_client"] = ee["cl_client"].fillna("Unknown")
    ee["slot_in_epoch"] = ee["slot"] % 32
    
    dfg0 = ee.groupby(["cl_client", "slot_in_epoch"])["validator"].count().reset_index().sort_values("validator", ascending=False)
    dfg0 = dfg0.reset_index(drop=True)
    dfg0["validator"] = dfg0["validator"] / (df.slot.nunique()/32)

    fig = go.Figure()

    # Add a bar for each label
    for ix, label in enumerate(ee.groupby("cl_client")["validator"].nunique().reset_index().sort_values("validator", ascending=False)["cl_client"].tolist()):
        df_label = dfg0[dfg0['cl_client'] == label]
        fig.add_trace(go.Bar(
            x=df_label['slot_in_epoch'],
            y=df_label['validator'],
            name=label,
            marker_color=COLORS2[ix % len(COLORS2)]
        ))

    # Update layout
    fig.update_layout(
        title=f'Missed/Failed {WHAT[0].upper() + WHAT[1:]} Votes over Slot In Epoch <span style="font-size: 16px;">({slot_to_day(min_max_slots[0])} - {slot_to_day(min_max_slots[1])})</span>',
        barmode='stack',
        legend_traceorder="normal",
        xaxis=dict(
            tickmode='linear',
            showgrid=True,
            gridcolor='lightgrey',
            title=f"slot in epoch",
        ),
        yaxis=dict(
            title=f"missed/failed {WHAT} votes",
            showgrid=True,
            gridcolor='lightgrey',
        ),
        height=550,
        width=1200,
        font=dict(
            family="Ubuntu Mono",
            size=FONTSIZE,
            color="black"
        ),
        plot_bgcolor = "#FFFFFF"
        #yaxis = dict(
        #    type="log"
        #)
    )
    return fig

fig = missed_over_clients_over_slots_in_epoch(df, clients_final)
fig.write_image(f"{IMAGE}/failed_missed_{WHAT}_votes_over_clclient_over_slot.png")
if SHOW:
    fig.show()
print("done")

In [ ]:
def missed_failed_with_timing_in_slot(_df):
    fontfamily = "Ubuntu Mono"
    df = _df.copy()
    df = combine_failed_and_missed_columns(df)
    df = pd.merge(df, block_timing, how="left", left_on="slot", right_on="slot").dropna()
    df["time_in_slot"] = df["time_in_slot"].apply(lambda x: x if x < 2 else 2)
    
    print(f"having {df.epoch.nunique()} epochs, {df.epoch.nunique()//225} days")
    share = get_share(df)    
    
    ee = df[["validator" ,"time_in_slot", "status"]]
    

    ee = ee.groupby(["time_in_slot", "status"])["validator"].count().reset_index().sort_values("time_in_slot", ascending=False)
    ee["time_in_slot_per"] = ee["time_in_slot"].apply(lambda x: share[x])
    ee["validator"] = ee["validator"] / ee["time_in_slot_per"]
    rr = set(ee[ee["time_in_slot_per"] >= 25].time_in_slot.tolist())
    ee = ee[ee["time_in_slot"].isin(rr)]
    ee = ee[ee["time_in_slot"] >= 0]
    ee = ee[ee["time_in_slot"] <= 10]
    ee.replace("failed", f"wrong {WHAT} vote", inplace=True)
    ee.replace("missed", f"missed/offline", inplace=True)
    ee.sort_values(["time_in_slot", "status"], ascending=[False, True], inplace=True)
    
    fig = px.bar(ee, x='time_in_slot', y='validator', color='status', title='Validator Status by Time in Slot',
                 labels={'time_in_slot': 'Time in Slot', 'validator': 'Validator', 'status': 'Status'},
                 barmode='stack')


    fig.update_layout(
        title = f'Missed/Failed {WHAT[0].upper() + WHAT[1:]} Votes over Seconds in Slot <span style="font-size: 16px;">({slot_to_day(df.slot.min())} - {slot_to_day(df.slot.max())})</span>',
        barmode='stack',
        legend_traceorder="normal",
        legend=dict(x=0, y=0.98, title=None),

        xaxis=dict(
            #tickmode='linear',
            #range=[0, 2],
             tickmode = 'array',
            tickvals = list(np.array(list(range(0,20)))/10) + [2],
            ticktext = list(np.array(list(range(0,20)))/10) + ["2+"],
            showgrid=True,
            gridcolor='lightgrey',
            title="seconds in slot (builder bid received)"
        ),
        yaxis=dict(
            title=f"missed/failed {WHAT} votes per slot",
            showgrid=True,
            gridcolor='lightgrey',
        ),
        height=550,
        width=1200,
        font=dict(
            family="Ubuntu Mono",
            size=FONTSIZE,
            color="black"
        ),
        plot_bgcolor = "#FFFFFF"
        #yaxis = dict(
        #    type="log"
        #)
    )
    share = {i: share[i] for i in share.keys() if i >= 0}
    share = {i: share[i] for i in share.keys() if i in rr}
    for i, size in enumerate(share.keys()):
        if (size*10+1) % 2 == 0:
            delta = 0.02
        else:
            delta = -0.02
        fig.add_annotation(
            text=f"n={share[size]:,}",
            xref="x",
            yref="paper",
            x=size,
            y=0.05+delta,
            showarrow=False,
            font=dict(
                family=fontfamily,
                size=FONTSIZE-4,
                color="black"
            )
        )
    return fig

fig = missed_failed_with_timing_in_slot(df)
fig.write_image(f"{IMAGE}/failed_missed_{WHAT}_votes_over_timing.png")
if SHOW:
    fig.show()
print("done")

In [ ]:
def missed_failed_with_timing_in_slot_per(_df):
    fontfamily = "Ubuntu Mono"
    df = _df.copy()
    df = combine_failed_and_missed_columns(df)
    df = pd.merge(df, block_timing, how="left", left_on="slot", right_on="slot").dropna()
    df["time_in_slot"] = df["time_in_slot"].apply(lambda x: x if x < 2 else 2)
    
    print(f"having {df.epoch.nunique()} epochs, {df.epoch.nunique()//225} days")
    share = get_share(df)   
    
    ee = df[["validator" ,"time_in_slot", "status"]]
    

    ee = ee.groupby(["time_in_slot", "status"])["validator"].count().reset_index().sort_values("time_in_slot", ascending=False)
    ee["time_in_slot_per"] = ee["time_in_slot"].apply(lambda x: share[x])
    ee["validator"] = ee["validator"] / ee["time_in_slot_per"]
    rr = set(ee[ee["time_in_slot_per"] >= 25].time_in_slot.tolist())
    ee = ee[ee["time_in_slot"].isin(rr)]
    ee = ee[ee["time_in_slot"] >= 0]
    ee = ee[ee["time_in_slot"] <= 10]
    ee.replace("failed", f"wrong {WHAT} vote", inplace=True)
    ee.replace("missed", f"missed/offline", inplace=True)
    ee.sort_values(["time_in_slot", "status"], ascending=[False, True], inplace=True)

    
    ee["per"] = ee["validator"] / 32_500 * 100
    fig = px.bar(ee, x='time_in_slot', y='per', color='status', title='Validator Status by Time in Slot',
                 labels={'time_in_slot': 'Time in Slot', 'validator': 'Validator', 'status': 'Status'},
                 barmode='stack')

    fig.add_hline(y=100, line=dict(color='red', dash='dash'), annotation_text='all validators', 
                  annotation_position='top right')

    fig.update_layout(
        title = f'Missed/Failed {WHAT[0].upper() + WHAT[1:]} Votes over Seconds in Slot <span style="font-size: 16px;">({slot_to_day(df.slot.min())} - {slot_to_day(df.slot.max())})</span>',
        barmode='stack',
        legend_traceorder="normal",
        legend=dict(x=0, y=0.98, title=None),

        xaxis=dict(
             tickmode = 'array',
            tickvals =  list(np.array(list(range(0,20)))/10) + [2],
            ticktext =  list(np.array(list(range(0,20)))/10) + ["2+"],
            showgrid=True,
            gridcolor='lightgrey',
            title="seconds in slot (builder bid received)"
        ),
        yaxis=dict(
            title=f"missed/failed {WHAT} votes per slot (%)",
            showgrid=True,
            gridcolor='lightgrey',
        ),
        height=550,
        width=1200,
        font=dict(
            family="Ubuntu Mono",
            size=FONTSIZE,
            color="black"
        ),
        plot_bgcolor = "#FFFFFF"
    )
    share = {i: share[i] for i in share.keys() if i >= 0}
    share = {i: share[i] for i in share.keys() if i in rr}
    for i, size in enumerate(share.keys()):
        if (size*10+1) % 2 == 0:
            delta = 0.02
        else:
            delta = -0.02
        fig.add_annotation(
            text=f"n={share[size]:,}",
            xref="x",
            yref="paper",
            x=size,
            y=0.05+delta,
            showarrow=False,
            font=dict(
                family=fontfamily,
                size=FONTSIZE-4,
                color="black"
            )
        )
    return fig

fig = missed_failed_with_timing_in_slot_per(df)
fig.write_image(f"{IMAGE}/failed_missed_{WHAT}_votes_over_timing_per.png")
if SHOW:
    fig.show()
print("done")

In [ ]:
def missed_failed_per_validator_per(df):
    _df = df.groupby("validator")["epoch"].nunique().reset_index().sort_values("epoch")
    _df = _df[_df["epoch"] > 1]
    distribution = sorted(_df.epoch)
    fig = px.histogram(distribution, nbins=400, histnorm='percent')

    fig.update_traces(name='Nr. of Validators', marker_color=COLORS[0])

    fig.update_layout(
        title=f'Missed/Failed {WHAT[0].upper() + WHAT[1:]} Votes Per Validator <span style="font-size: 16px;">({slot_to_day(df.slot.min())} - {slot_to_day(df.slot.max())})</span>',
        xaxis_title='slot in epoch',
        barmode='stack',
        legend_traceorder="normal",
        legend_title_text='',
        showlegend=False,
        xaxis=dict(
            showgrid=True,
            gridcolor='lightgrey',
            title=f"nr. of missed/failed {WHAT} votes per validator",
            range=[1, max(distribution)],
            tickvals=[1] + list(range(0, max(distribution) + 1, 250)),
            ticktext=[1] + list(range(0, max(distribution) + 1, 250))
        ),
        yaxis=dict(
            title="% of validators",
            showgrid=True,
            gridcolor='lightgrey',
            type="log"
        ),
        height=550,
        width=1200,
        font=dict(
            family="Ubuntu Mono",
            size=FONTSIZE,
            color="black"
        ),
        plot_bgcolor="#FFFFFF"
    )
    return fig
fig = missed_failed_per_validator_per(df)
fig.write_image(f"{IMAGE}/failed_missed_{WHAT}_per_validator_dist_per.png")
if SHOW:
    fig.show()
print("done")



In [ ]:
###########################################

In [ ]:
#df = prepare_failed_with_size(df, only_failed=True, overwrite=False)
#df = df[df["size_compressed_total"] < 1_000_000]
#df["size_compressed_total"] = df["size_compressed_total"] / 1024**2
##six = df[df["blobs"] == 128*1024*6].copy()
#hm = df[df["validator"] >= 30_000].copy()
##six

In [ ]:
#qa = f"""
#    SELECT DISTINCT
#        slot, min(event_date_time) event_date_time
#    FROM default.beacon_api_eth_v1_events_block
#    WHERE
#        slot_start_date_time > NOW() - INTERVAL '100 day'
#        and slot >= {df.slot.min()}
#    group by slot
#    """
#
#res = request_query(qa, ["slot", "event_date_time"])
#def parse_datetime(event_time_str):
#    return datetime.strptime(event_time_str, '%Y-%m-%d %H:%M:%S.%f').replace(tzinfo=pytz.UTC).timestamp()*1000
#
#res['event_date_time_unix'] = res['event_date_time'].apply(lambda x: parse_datetime(x))
#res["seconds_in_slot"] = res.apply(lambda x: get_time_in_curr_slot(x["event_date_time_unix"], x["slot"]), axis=1)
#
#high = res[res["slot"].isin((hm["slot"]).tolist())]
#low = res[~res["slot"].isin((hm["slot"]).tolist())]

In [ ]:
#gg = pd.merge(df, res, how="left", left_on="slot", right_on="slot")
#gg["blobs"] = gg["blobs"] / (1024*128)
#gg

In [ ]:
#def tt(_df):
#    
#    df = _df.copy()
#    df = prepare_failed_with_size(df, only_failed=False, overwrite=True)
#    #df = df[df["size_compressed_total"] < 1_000_000]
#    df["size_compressed_total"] = df["size_compressed_total"] / 1024**2
#    #six = df[df["blobs"] == 128*1024*6].copy()
#    hm = df[df["validator"] >= 30_000].copy()
#    
#    def calculate_histogram(data, bins, x_range):
#        hist, bin_edges = np.histogram(data, bins=bins, range=x_range)
#        # Normalize to percentage
#        total_count = np.sum(hist)
#        hist_normalized = (hist / total_count) * 100
#        return bin_edges, hist_normalized
#    
#    qa = f"""
#        SELECT DISTINCT
#            slot, min(event_date_time) event_date_time
#        FROM default.beacon_api_eth_v1_events_block
#        WHERE
#            slot_start_date_time > NOW() - INTERVAL '100 day'
#            and slot >= {df.slot.min()}
#        group by slot
#        """
#
#    res = request_query(qa, ["slot", "event_date_time"])
#    
#    qa1 = f"""
#        SELECT DISTINCT
#            slot
#        FROM default.canonical_beacon_block
#        WHERE
#            slot_start_date_time > NOW() - INTERVAL '100 day'
#            and slot >= {df.slot.min()}
#    """
#    res_can = request_query(qa1, ["slot"])
#    res = res[res["slot"].isin(res_can.slot.unique())]
#    return res, df
#
#o, _df = tt(df)
#
#qa = f"""
#SELECT DISTINCT
#    min(event_date_time) event_date_time, slot, kzg_commitment
#FROM (SELECT DISTINCT 
#        event_date_time, 
#        slot, 
#        kzg_commitment 
#    FROM default.beacon_api_eth_v1_events_blob_sidecar
#    WHERE
#        event_date_time > NOW() - INTERVAL '100 day'
#        AND meta_network_name = 'mainnet'
#        AND slot >= {o.slot.min()}
#    )
#GROUP BY slot, kzg_commitment
#"""
#blob_times = request_query(qa, ["event_date_time", "slot", "kzg_commitment"])
#
#blob_times = blob_times.groupby("slot")["event_date_time"].max().reset_index()
#blob_times.columns = ["slot", "last_blob_timing"]
#
#res = pd.merge(o, blob_times, how="left", left_on="slot", right_on="slot")
#res = res.dropna()
#
#def parse_datetime(event_time_str):
#    return datetime.strptime(event_time_str, '%Y-%m-%d %H:%M:%S.%f').replace(tzinfo=pytz.UTC).timestamp()*1000
#
#res['event_date_time_unix'] = res['event_date_time'].apply(lambda x: parse_datetime(x))
#res['last_blob_timing_unix'] = res['last_blob_timing'].apply(lambda x: parse_datetime(x))
#res["seconds_in_slot"] = res.apply(lambda x: get_time_in_curr_slot(x["event_date_time_unix"], x["slot"]), axis=1)
#res["seconds_in_slot2"] = res.apply(lambda x: get_time_in_curr_slot(x["last_blob_timing_unix"], x["slot"]), axis=1)
#
#gg = pd.merge(_df, res, left_on="slot", right_on="slot")
#gg["blobs"] = gg["blobs"] / (1024*128)
#gg["block_earlier"] = gg["seconds_in_slot"] < gg["seconds_in_slot2"]
#gg.groupby("block_earlier")["slot"].count().reset_index()

In [ ]:
def hist_late_performer_blobs(_df):
    
    df = _df.copy()
    df = prepare_failed_with_size(df, only_failed=True, overwrite=False)
    #df = df[df["size_compressed_total"] < 1_000_000]
    df["size_compressed_total"] = df["size_compressed_total"] / 1024**2
    #six = df[df["blobs"] == 128*1024*6].copy()
    hm = df[df["validator"] >= 30_000].copy()
    
    def calculate_histogram(data, bins, x_range):
        hist, bin_edges = np.histogram(data, bins=bins, range=x_range)
        # Normalize to percentage
        total_count = np.sum(hist)
        hist_normalized = (hist / total_count) * 100
        return bin_edges, hist_normalized
    
    qa = f"""
        SELECT DISTINCT
            slot, min(event_date_time) event_date_time
        FROM default.beacon_api_eth_v1_events_block
        WHERE
            slot_start_date_time > NOW() - INTERVAL '100 day'
            and slot >= {df.slot.min()}
        group by slot
        """

    res = request_query(qa, ["slot", "event_date_time"])
    
    qa1 = f"""
        SELECT DISTINCT
            slot
        FROM default.canonical_beacon_block
        WHERE
            slot_start_date_time > NOW() - INTERVAL '100 day'
            and slot >= {df.slot.min()}
    """
    res_can = request_query(qa1, ["slot"])
    res = res[res["slot"].isin(res_can.slot.unique())]
    def parse_datetime(event_time_str):
        return datetime.strptime(event_time_str, '%Y-%m-%d %H:%M:%S.%f').replace(tzinfo=pytz.UTC).timestamp()*1000

    res['event_date_time_unix'] = res['event_date_time'].apply(lambda x: parse_datetime(x))
    res["seconds_in_slot"] = res.apply(lambda x: get_time_in_curr_slot(x["event_date_time_unix"], x["slot"]), axis=1)
    
    gg = pd.merge(df, res, how="left", left_on="slot", right_on="slot")
    gg["blobs"] = gg["blobs"] / (1024*128)


    blobs = dict()
    for i in range(0,7):
        j = gg[gg["blobs"] == i]
        print(j.seconds_in_slot.mean())
        blobs[i] = calculate_histogram(j.seconds_in_slot, 30, (0,4.1))

    fig = go.Figure()

    for i, c in zip([1, 6], [0, 1]): # zip([0, 1, 6], [0, 1, 2]):
        fig.add_trace(go.Scatter(
            x=blobs[i][0],
            y=blobs[i][1],
            fill='tozeroy',
            mode='lines',
            name=f"blocks with {i} blob(s)",
            line=dict(color=COLORS3[c]),
            opacity=1/(i+1)
        ))

    fig.update_layout(
        title=f'Blocks First Seen Timing <span style="font-size: 16px;">({slot_to_day(df.slot.min())} - {slot_to_day(df.slot.max())})</span>',
        yaxis_title='%',
        xaxis_title='seconds in slot',
        barmode='stack',
        legend_traceorder="normal",
        xaxis=dict(
            tickmode='linear',
            showgrid=True,
            gridcolor='lightgrey',
            range=[0, 4.01]
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgrey',
        ),
        height=IMAGE_HEIGHT,
        width=1200,
        font=dict(
            family="Ubuntu Mono",
            size=FONTSIZE,
            color="black"
        ),
        legend=dict(
            x=1,
            y=1,
            xanchor='right',
            yanchor='top'
        ),
        plot_bgcolor = "#FFFFFF"
        #yaxis = dict(
        #    type="log"
        #)
    )

    return fig

if WHAT == "head":
    fig = hist_late_performer_blobs(df)
    fig.write_image(f"{IMAGE}/hist_late_performer_blobs.png")
    if SHOW:
        fig.show()

In [ ]:
#def calculate_kde(data, x_range):
#    kde = gaussian_kde(data)
#    x = np.linspace(x_range[0], x_range[1], 1000)
#    y = kde(x)
#    # Normalize the KDE to ensure the area under the curve is 1
#    area = np.trapz(y, x)
#    y_normalized = y / area * 100  # Scale to 100%
#    return x, y_normalized
#
#x_jj, y_jj = calculate_kde(low['seconds_in_slot'], (0, 10))
#x_rr, y_rr = calculate_kde(high['seconds_in_slot'], (0, 10))

In [ ]:
def hist_late_performer(_df):   
    df = _df.copy()
    
    df = prepare_failed_with_size(df, only_failed=True, overwrite=False)
    #df = df[df["size_compressed_total"] < 1_000_000]
    df["size_compressed_total"] = df["size_compressed_total"] / 1024**2
    #hm = df[df["blobs"] == 128*1024*6].copy()
    hm = df[df["validator"] >= 30_000].copy()
    
    qa = f"""
        SELECT DISTINCT
            slot, min(event_date_time) event_date_time
        FROM default.beacon_api_eth_v1_events_block
        WHERE
            slot_start_date_time > NOW() - INTERVAL '100 day'
            and slot >= {df.slot.min()}
        group by slot
        """

    res = request_query(qa, ["slot", "event_date_time"])
    def parse_datetime(event_time_str):
        return datetime.strptime(event_time_str, '%Y-%m-%d %H:%M:%S.%f').replace(tzinfo=pytz.UTC).timestamp()*1000

    res['event_date_time_unix'] = res['event_date_time'].apply(lambda x: parse_datetime(x))
    res["seconds_in_slot"] = res.apply(lambda x: get_time_in_curr_slot(x["event_date_time_unix"], x["slot"]), axis=1)
    
    high = res[res["slot"].isin((hm["slot"]).tolist())]
    low = res[~res["slot"].isin((hm["slot"]).tolist())]
    
    def calculate_histogram(data, bins, x_range):
        hist, bin_edges = np.histogram(data, bins=bins, range=x_range)
        # Normalize to percentage
        total_count = np.sum(hist)
        hist_normalized = (hist / total_count) * 100
        return bin_edges, hist_normalized

    x_jj, y_jj = calculate_histogram(low['seconds_in_slot'], 50, (0, 10.1))
    x_rr, y_rr = calculate_histogram(high['seconds_in_slot'], 50, (0, 10.1))


    fig = go.Figure()

    # Add KDE plot for the first dataset
    fig.add_trace(go.Scatter(
        x=x_jj,
        y=y_jj,
        fill='tozeroy',
        mode='lines',
        name='Rest',
        line=dict(color=COLORS3[0]),
        opacity=0.5
    ))
    fig.add_trace(go.Scatter(
            x=x_rr,
            y=y_rr,
            fill='tozeroy',
            mode='lines',
            name=f'Weak Blocks (n={len(high)})',
            line=dict(color=COLORS3[1]),
            opacity=0.5
        ))

    fig.update_layout(
        title=f'Blocks First Seen Timing - Weak Blocks vs Rest <span style="font-size: 16px;">({slot_to_day(df.slot.min())} - {slot_to_day(df.slot.max())})</span>',
        yaxis_title='%',
        xaxis_title='seconds in slot',
        barmode='stack',
        legend_traceorder="normal",
        xaxis=dict(
            tickmode='linear',
            showgrid=True,
            gridcolor='lightgrey',
            range=[0, 10.01]
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgrey',
        ),
        height=IMAGE_HEIGHT,
        width=1200,
        font=dict(
            family="Ubuntu Mono",
            size=FONTSIZE,
            color="black"
        ),
        legend=dict(
            x=1,
            y=1,
            xanchor='right',
            yanchor='top'
        ),
        plot_bgcolor = "#FFFFFF"
        #yaxis = dict(
        #    type="log"
        #)
    )

    return fig

if WHAT == "head":
    fig = hist_late_performer(df)
    fig.write_image(f"{IMAGE}/hist_late_performer.png")
    if SHOW:
        fig.show()

In [ ]:
def hist_late_performer_mevboost(_df):   
    df = _df.copy()
    
    df = prepare_failed_with_size(df, only_failed=True, overwrite=False)
    #df = df[df["size_compressed_total"] < 1_000_000]
    df["size_compressed_total"] = df["size_compressed_total"] / 1024**2
    hm = df[df["blobs"] == 128*1024*6].copy()
    hm = hm[hm["validator"] >= 30_000].copy()
    
    qa = f"""
        SELECT DISTINCT
            slot, min(event_date_time) event_date_time
        FROM default.beacon_api_eth_v1_events_block
        WHERE
            slot_start_date_time > NOW() - INTERVAL '100 day'
            and slot >= {df.slot.min()}
        group by slot
        """

    res = request_query(qa, ["slot", "event_date_time"])
    def parse_datetime(event_time_str):
        return datetime.strptime(event_time_str, '%Y-%m-%d %H:%M:%S.%f').replace(tzinfo=pytz.UTC).timestamp()*1000

    res['event_date_time_unix'] = res['event_date_time'].apply(lambda x: parse_datetime(x))
    res["seconds_in_slot"] = res.apply(lambda x: get_time_in_curr_slot(x["event_date_time_unix"], x["slot"]), axis=1)
    
    
    mevboost = pandas_gbq.read_gbq(f"""
        SELECT distinct slot FROM `ethereum-data-nero.eth.mevboost` WHERE slot >= {df.slot.min()}
        order by slot
    """)
    mevboost.to_parquet(f"{DATA}/mevboost.parquet", index=False)

    mb = res[res["slot"].isin(mevboost["slot"])]
    nmb = res[~res["slot"].isin(mevboost["slot"])]
    
    def calculate_histogram(data, bins, x_range):
        hist, bin_edges = np.histogram(data, bins=bins, range=x_range)
        # Normalize to percentage
        total_count = np.sum(hist)
        hist_normalized = (hist / total_count) * 100
        return bin_edges, hist_normalized

    x_jj, y_jj = calculate_histogram(mb['seconds_in_slot'], 50, (0, 4.1))
    x_rr, y_rr = calculate_histogram(nmb['seconds_in_slot'], 50, (0, 4.1))


    fig = go.Figure()

    # Add KDE plot for the first dataset
    fig.add_trace(go.Scatter(
        x=x_jj,
        y=y_jj,
        fill='tozeroy',
        mode='lines',
        name='MEV-Boost',
        line=dict(color=COLORS3[0]),
        opacity=0.5
    ))
    fig.add_trace(go.Scatter(
            x=x_rr,
            y=y_rr,
            fill='tozeroy',
            mode='lines',
            name=f'Local Building',
            line=dict(color=COLORS3[1]),
            opacity=0.5
        ))

    fig.update_layout(
        title=f'Blocks First Seen Timing - MEV-Boost vs Local Building <span style="font-size: 16px;">({slot_to_day(df.slot.min())} - {slot_to_day(df.slot.max())})</span>',
        yaxis_title='%',
        xaxis_title='seconds in slot',
        barmode='stack',
        legend_traceorder="normal",
        xaxis=dict(
            tickmode='linear',
            showgrid=True,
            gridcolor='lightgrey',
            range=[0, 4.01]
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgrey',
        ),
        height=IMAGE_HEIGHT,
        width=1200,
        font=dict(
            family="Ubuntu Mono",
            size=FONTSIZE,
            color="black"
        ),
        legend=dict(
            x=1,
            y=1,
            xanchor='right',
            yanchor='top'
        ),
        plot_bgcolor = "#FFFFFF"
        #yaxis = dict(
        #    type="log"
        #)
    )

    return fig

if WHAT == "head":
    fig = hist_late_performer_mevboost(df)
    fig.write_image(f"{IMAGE}/hist_late_performer_mevboost.png")
    if SHOW:
        fig.show()

In [ ]:
def hist_mevboost_relays(_df):
    tt = _df.copy()

    tt = prepare_failed_with_size(tt, only_failed=True, overwrite=False)
    #tt = tt[tt["size_compressed_total"] < 1_000_000]
    tt["size_compressed_total"] = tt["size_compressed_total"] / 1024**2
    hm = tt[tt["blobs"] == 128*1024*6].copy()
    hm = hm[hm["validator"] >= 30_000].copy()

    qa = f"""
        SELECT DISTINCT
            slot, min(event_date_time) event_date_time
        FROM default.beacon_api_eth_v1_events_block
        WHERE
            slot_start_date_time > NOW() - INTERVAL '100 day'
            and slot >= {tt.slot.min()}
        group by slot
        """

    res = request_query(qa, ["slot", "event_date_time"])
    
    qa1 = f"""
        SELECT DISTINCT
            slot
        FROM default.canonical_beacon_block
        WHERE
            slot_start_date_time > NOW() - INTERVAL '100 day'
            and slot >= {df.slot.min()}
    """
    res_can = request_query(qa1, ["slot"])
    res = res[res["slot"].isin(res_can.slot.unique())]
    def parse_datetime(event_time_str):
        return datetime.strptime(event_time_str, '%Y-%m-%d %H:%M:%S.%f').replace(tzinfo=pytz.UTC).timestamp()*1000

    res['event_date_time_unix'] = res['event_date_time'].apply(lambda x: parse_datetime(x))
    res["seconds_in_slot"] = res.apply(lambda x: get_time_in_curr_slot(x["event_date_time_unix"], x["slot"]), axis=1)


    mevboost = pandas_gbq.read_gbq(f"""
        SELECT distinct slot, relay FROM `ethereum-data-nero.eth.mevboost` WHERE slot >= {tt.slot.min()}
        order by slot
    """)
    tt = pd.merge(res, mevboost, how="left", left_on="slot", right_on="slot")

    def calculate_histogram(data, bins, x_range):
            hist, bin_edges = np.histogram(data, bins=bins, range=x_range)
            # Normalize to percentage
            total_count = np.sum(hist)
            hist_normalized = (hist / total_count) * 100
            return bin_edges, hist_normalized

    relays = dict()
    for i in tt.relay.unique():
        j = tt[tt["relay"] == i]
        print(i, j.seconds_in_slot.mean())
        relays[i] = calculate_histogram(j.seconds_in_slot, 30, (0,4.1))

    fig = go.Figure()

    col = [
        '#377eb8',
        '#ff7f00',
     '#33a02c',
     '#ff7f7f',
     '#8dd3c7',
     '#ffffb3',
     '#bebada',
     '#fb8072',
     '#80b1d3',
     '#fdb462'
    ]
    for ix, i in enumerate(["ultrasound", "bloxroute (max profit)", "titan"]):
        fig.add_trace(go.Scatter(
            x=relays[i][0],
            y=relays[i][1],
            fill='tozeroy',
            mode='lines',
            name=f"{i}",
            line=dict(color=col[ix]),
            opacity=0.9
        ))

    fig.update_layout(
        title=f'Blocks First Seen Timing <span style="font-size: 16px;">({slot_to_day(df.slot.min())} - {slot_to_day(df.slot.max())})</span>',
        yaxis_title='%',
        xaxis_title='seconds in slot',
        barmode='stack',
        legend_traceorder="normal",
        xaxis=dict(
            tickmode='linear',
            showgrid=True,
            gridcolor='lightgrey',
            range=[0, 4.01]
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgrey',
        ),
        height=IMAGE_HEIGHT,
        width=1200,
        font=dict(
            family="Ubuntu Mono",
            size=FONTSIZE,
            color="black"
        ),
        legend=dict(
            x=1,
            y=1,
            xanchor='right',
            yanchor='top'
        ),
        plot_bgcolor = "#FFFFFF"
        #yaxis = dict(
        #    type="log"
        #)
    )
    return fig

if WHAT == "head":
    fig = hist_mevboost_relays(df)
    fig.write_image(f"{IMAGE}/hist_mevboost_relays.png")
    if SHOW:
        fig.show()

In [ ]:
def missed_failed_and_size_scatter(_df):
    #df = df.sample(1_000_000)
    df = _df.copy()
    df = prepare_failed_with_size(df, only_failed=True, overwrite=False)
    df = df[df["size_compressed_total"] < 1_000_000]
    df["size_compressed_total"] = df["size_compressed_total"] / 1024**2

    fig = go.Figure()

    fontfamily = "Ubuntu Mono"

    # Adding traces for each slot
    #fig.add_trace(go.Scatter(
    #    x=df['slot'],
    #    y=df['size'],
    #    mode='markers',
    #    name='Size',
    #    marker=dict(size=8, color='blue')
    #))
    #

    fig.add_trace(go.Scatter(
        x=df['validator'],
        y=df['size_compressed_total'],
        mode='markers',
        name='Size Compressed',
        marker=dict(size=2, color=COLORS[0])  # Adjust size and color
    ))

    fig.update_layout(
        title={
            'text': f'Failed {WHAT[0].upper()+WHAT[1:]} Votes vs. Block Size <span style="font-size: 16px;">({slot_to_day(_df.slot.min())} - {slot_to_day(_df.slot.max())})</span>',
            #'y':0.9,
            #'x':0.5,
            #'xanchor': 'center',
            #'yanchor': 'top',
            'font': dict(family=fontfamily, size=24, color='#333')
        },
        xaxis_title=f"failed votes per slot",
        yaxis_title="compressed block size (incl. blobs)",
        font=dict(
            family=fontfamily,
            size=FONTSIZE,
            color="black"
        ),
        paper_bgcolor='white',
        plot_bgcolor='white',
        xaxis=dict(
            showgrid=True,
            gridwidth=1, 
            gridcolor='LightGray',
            zerolinecolor='gray',
            #type="log"
        ),
        yaxis=dict(
            showgrid=True,
            gridwidth=1, 
            gridcolor='LightGray',
            zerolinecolor='gray'
        ),
        height=550,
        width=1200,    
    )

    blob_sizes = [i * 128/1024 for i in range(7)]
    df["blobs"] = df["blobs"] / 1024**2
    obs_counts = df.groupby("blobs")["slot"].count().to_dict()

    texts = []
    for i, size in enumerate(blob_sizes):
        print(len(str(obs_counts[size])))
        if len(str(obs_counts[size])) == 5:
            texts.append(f"{'n=':>8}{obs_counts[size]:<6,}")
        else:
            texts.append(f"{'n=':<4}{obs_counts[size]:<6,}")


    for i, (size, text) in enumerate(zip(blob_sizes, texts)):
        fig.add_annotation(
            text=text,
            xref="paper",
            yref="y",
            x=1.05,
            y=size + 80/1024,
            showarrow=False,
            font=dict(
                family=fontfamily,
                size=FONTSIZE-2,
                color="black"
            )
        )

    return fig

if WHAT != "source":
    fig = missed_failed_and_size_scatter(df)
    fig.write_image(f"{IMAGE}/failed_missed_{WHAT}_size_scatter.png")
    #if SHOW:
        #fig.show()
    print("done")

In [ ]:
def missed_failed_boxplots(_df):
    df = prepare_failed_with_size(_df.copy(), only_failed=True if WHAT != "source" else False).dropna()
    df["nr_blobs"] = df["blobs"]/(128*1024)
    
    def percentile(n):
        def percentile_(x):
            return np.percentile(x, n)
        percentile_.__name__ = 'percentile_%s' % n
        return percentile_

    aggr = {
        "validator": ["mean", "median", "min", percentile(5), percentile(95), "max"]
    }

    fig = go.Figure()

    result = df.groupby("nr_blobs").agg(aggr).reset_index()
    result.columns = ['_'.join(col).strip() for col in result.columns.values]
    print(result["validator_percentile_95"].max())

    for nr_blob in df['nr_blobs'].unique():
        fig.add_trace(go.Box(
            x=[nr_blob] * df[df['nr_blobs'] == nr_blob].shape[0],
            y=df[df['nr_blobs'] == nr_blob]['validator'],
            name=str(nr_blob),
                boxpoints=False, # no data points

            marker_color=COLORS[1],  # Set box line color
            #boxmean='sd',
            line_color = COLORS[0]
        ))

    # Update the layout for better readability
    fig.update_layout(
        title=f'Missed Head Votes over Blobs per Block <span style="font-size: 16px;">({slot_to_day(df.slot.min())} - {slot_to_day(df.slot.max())})</span>',
        xaxis_title="number of blobs",
        showlegend=False,
        font=dict(
            family="Ubuntu Mono",
            size=FONTSIZE,
            color="black"
        ),
        yaxis=dict(
            range=[0, 180],
            title="missed/failed head votes",
            showgrid=True,
            gridcolor='lightgrey'
        ),
        xaxis=dict(
            showgrid=True,
            gridcolor='lightgrey'
        ),
        paper_bgcolor='white',
        plot_bgcolor='white',
        height=550,
        width=1200,
        
    )
    obs_counts = df.groupby("nr_blobs")["slot"].count().to_dict()
    texts = []
    for i, size in enumerate(df['nr_blobs'].unique()):
        print(len(str(obs_counts[size])))
        if len(str(obs_counts[size])) == 5:
            texts.append(f"{'n=':>8}{obs_counts[size]:<6,}")
        else:
            texts.append(f"{'n=':<4}{obs_counts[size]:<6,}")
            
            
    for i, (size, text) in enumerate(zip(df['nr_blobs'].unique(), texts)):
        fig.add_annotation(
            text=text,
            xref="x",
            yref="y",
            x=i+0.2,
            y=170,
            showarrow=False,
            font=dict(
                family="Ubuntu Mono",
                size=FONTSIZE-1,
                color="black"
            )
        )
    return fig

if WHAT == "head":
    fig = missed_failed_boxplots(df)
    fig.write_image(f"{IMAGE}/failed_missed_{WHAT}_size_boxplot.png")
    #if SHOW:
    #    fig.show()
    print("done")

In [ ]:
def show_examples_for_late_inclusion(_df):
    gg = pd.DataFrame()
    for ix, i in enumerate(sorted(_df.epoch.unique())):
        #print(f"{ix}/{l}", end="\r")
        #if ix > 3:
        #    break
        g = pd.read_parquet(f"{DATA}/correct_{WHAT}/{i}.parquet").drop_duplicates()
        g = g[g["validators"] != 999999999999999]
        g = pd.merge(g, clients_final, how="left", left_on="validators", right_on="validator_id")
        g["cl_client"] = g["cl_client"].fillna("Unknown")
        #g = g.groupby(["cl_client", "distance"])["slot"].count().reset_index()
        gg = pd.concat([gg, g], ignore_index=True)
        if ix > 15:
            print("examples with late inclusion")
            print(g[g["distance"] > 30].groupby(["slot", "distance"])["validators"].count().reset_index().sort_values("validators", ascending=False))
            break
    return gg
       
if WHAT == "head":
    hh = show_examples_for_late_inclusion(df)

In [ ]:
if WHAT == "head":
    try:
        raise
        proposers = pd.read_parquet(f"{DATA}/proposers.parquet")
        print("read locally")
    except:
        proposers = pandas_gbq.read_gbq(f"""
            SELECT distinct slot, validator_id FROM `ethereum-data-nero.ethdata.beaconchain` WHERE slot >= {df.slot.min()}
            order by slot
        """)
        proposers.to_parquet(f"{DATA}/proposers.parquet", index=False)

    proposers.columns = ["slot", "proposer_id"]
    clients_final["validator_id"] = clients_final["validator_id"].astype(int)
    proposers["proposer_id"] = proposers["proposer_id"].astype(int)
    ee = pd.merge(proposers, clients_final, how="left", left_on="proposer_id", right_on="validator_id")
    ee

In [ ]:
def correct_delay_proposers(df):
    startslot = min(df.slot)
    endslot = max(df.slot)
    gg = list()
    l = len(df.epoch.unique())
    try:
        proposers = pd.read_parquet(f"{DATA}/proposers.parquet").drop_duplicates()
        print("read locally")
    except:
        proposers = pandas_gbq.read_gbq(f"""
            SELECT distinct slot, validator_id FROM `ethereum-data-nero.ethdata.beaconchain` WHERE slot >= {df.slot.min()}
            order by slot
        """)
        proposers.to_parquet(f"{DATA}/proposers.parquet", index=False)

    proposers.columns = ["slot", "proposer_id"]
    clients_final["validator_id"] = clients_final["validator_id"].astype(int)
    proposers["proposer_id"] = proposers["proposer_id"].astype(int)
    ee = pd.merge(proposers, clients_final, how="left", left_on="proposer_id", right_on="validator_id")
    for ix, i in enumerate(sorted(df.epoch.unique(), reverse=True)):
        print(f"{ix}/{l}", end="\r")
        #if ix > 10:
        #    break
        g = pd.read_parquet(f"{DATA}/correct_{WHAT}/{i}.parquet").drop_duplicates()
        g = g[g["validators"] != 999999999999999]
        g = pd.merge(g, clients_final, how="left", left_on="validators", right_on="validator_id")
        g["cl_client"] = g["cl_client"].fillna("Unknown")
        #g = g.groupby(["cl_client", "distance"])["slot"].count().reset_index()
        #gg = pd.concat([gg, g], ignore_index=True)

        g['re-included'] = True
        min_distance_idx = g.groupby(['slot', 'validators'])['distance'].idxmin()
        g.loc[min_distance_idx, 're-included'] = False
        #print(len(g[g["re-included"] == True]))
        g["sp"] = g["slot"] + g["distance"]
        g = pd.merge(g[g["re-included"] == True], ee, left_on="sp", right_on="slot")
        #print(g)
        #g = pd.merge(g, ee, left_on="sp", right_on="slot")
        g = g.groupby(["cl_client_y", "distance"])["slot_x"].count().reset_index()
        g.columns=["cl_client", "distance", "slot"]
        gg.append(g)
        
    gg = pd.concat(gg, ignore_index=True)
    gg.to_parquet(f"{DATA}/{TMP}/correct_delay_proposers.parquet", index=False)
    
    gg = gg.groupby(["cl_client", "distance"])["slot"].sum().reset_index()

    gg["per"] = gg["slot"]/ gg["slot"].sum() * 100
    
    #gg["distance"] = gg["distance"] -1


    fig = go.Figure()

    for ix, i in enumerate(gg.cl_client.unique()):
        _gg = gg[gg["cl_client"] == i].copy()
        fig.add_trace(go.Bar(
            x= _gg[_gg["distance"] != 1]["distance"],
            y= _gg[_gg["distance"] != 1]["per"],
            marker_color=COLORS[ix],
            name=f'{i}',
        ))
    fig.update_layout(
            title_text=f'Re-Submitted Attestation Inclusion Delay <span style="font-size: 16px;">({slot_to_day(startslot)} - {slot_to_day(endslot)})</span>',
            legend=dict(x=0.87, y=1),
            barmode='stack',
            legend_traceorder="reversed",
            xaxis=dict(
                tickmode='linear',
                showgrid=True,
                gridcolor='lightgrey',
                title="delay in slots",
                range=[1.5, 63]
            ),
            yaxis=dict(
                title="% of attestations",
                showgrid=True,
                gridcolor='lightgrey',
                #type="log",
                #range=[0,1]

            ),
            height=550,
            width=1200,
            font=dict(
                family="Ubuntu Mono",
                size=FONTSIZE,
                color="black"
            ),
            plot_bgcolor = "#FFFFFF"

        )
    #fig.add_annotation(
    #    text=f'0 slot inclusion: {gg[gg["distance"] == 0]["per"].sum():,.2f}%',
    #    xref="paper",
    #    yref="paper",
    #    x=1,
    #    y=1,
    #    showarrow=False,
    #    font=dict(
    #        family="Ubuntu Mono",
    #        size=18,
    #        color="black"
    #    )
    #)
    return fig
if WHAT == "head":
    fig = correct_delay_proposers(df)
    fig.write_image(f"{IMAGE}/correct_{WHAT}_delay_clients_proposers.png")
    if SHOW:
        fig.show()
    print("done")

In [ ]:
def correct__delay_reinclusion(df):
    startslot = 9271808
    endslot = max(df.slot)
    gg = []
    l = len(df.epoch.unique())
    for ix, i in enumerate(sorted(df.epoch.unique())):
        print(f"{ix}/{l}", end="\r")
        #if ix > 13:
        #    break
        g = pd.read_parquet(f"{DATA}/correct_{WHAT}/{i}.parquet").drop_duplicates()
        g = g[g["validators"] != 999999999999999]
        #g = g.groupby(["cl_client", "distance"])["slot"].count().reset_index()
        g['re-included'] = "reinclusion"
        min_distance_idx = g.groupby(['slot', 'validators'])['distance'].idxmin()
        g.loc[min_distance_idx, 're-included'] = "first inclusion"
        g = g.groupby(["re-included", "distance"])["slot"].count().reset_index()
        gg.append(g)
        
    gg = pd.concat(gg, ignore_index=True)
    gg.to_parquet(f"{DATA}/{TMP}/correct__delay_reinclusion.parquet", index=False)

    gg = gg.groupby(["re-included", "distance"])["slot"].sum().reset_index()
    
    gg["per"] = gg["slot"]/ gg["slot"].sum() * 100
    
    #gg["distance"] = gg["distance"] -1


    fig = go.Figure()

    for ix, i in enumerate(gg["re-included"].unique()):
        _gg = gg[gg["re-included"] == i]
        fig.add_trace(go.Bar(
            x= _gg[_gg["distance"] != 1]["distance"],
            y= _gg[_gg["distance"] != 1]["per"],
            marker_color=COLORS[ix],
            name=f'{i}',
        ))
    fig.update_layout(
        title_text=f'Attestation Inclusion Delay <span style="font-size: 14px;">({slot_to_day(startslot)} - {slot_to_day(endslot)})</span>',
        legend=dict(x=0.82, y=0.9),
        barmode='stack',
        legend_traceorder="reversed",
        xaxis=dict(
            tickmode='linear',
            showgrid=True,
            gridcolor='lightgrey',
            title="delay in slots",
            range=[1.5, 63]
        ),
        yaxis=dict(
            title="% of attestations",
            showgrid=True,
            gridcolor='lightgrey',
            #type="log",
            #range=[0,1]

        ),
        height=550,
        width=1200,
        font=dict(
            family="Ubuntu Mono",
            size=18,
            color="black"
        ),
        plot_bgcolor = "#FFFFFF"

    )
    fig.add_annotation(
        text=f'1 slot inclusion: {gg[gg["distance"] == 1]["per"].sum():,.2f}%',
        xref="paper",
        yref="paper",
        x=1,
        y=1,
        showarrow=False,
        font=dict(
            family="Ubuntu Mono",
            size=18,
            color="black"
        )
    )
    return fig
if WHAT == "head":
    fig = correct__delay_reinclusion(df)
    fig.write_image(f"{IMAGE}/correct_{WHAT}_delay_reinclusion.png")
    if SHOW:
        fig.show()
    print("done")

In [ ]:
def correct__delay_reinclusion_per(df):
    startslot = 9271808
    endslot = max(df.slot)
    gg = []
    l = len(df.epoch.unique())
    for ix, i in enumerate(sorted(df.epoch.unique())):
        print(f"{ix}/{l}", end="\r")
        #if ix > 130:
        #    break
        g = pd.read_parquet(f"{DATA}/correct_{WHAT}/{i}.parquet").drop_duplicates()
        g = g[g["validators"] != 999999999999999]
        #g = g.groupby(["cl_client", "distance"])["slot"].count().reset_index()
        g['re-included'] = "reinclusion"
        min_distance_idx = g.groupby(['slot', 'validators'])['distance'].idxmin()
        g.loc[min_distance_idx, 're-included'] = "first inclusion"
        g = g.groupby(["re-included", "distance"])["slot"].count().reset_index()
        gg.append(g)
        
    gg = pd.concat(gg, ignore_index=True)

    gg = gg.groupby(["re-included", "distance"])["slot"].sum().reset_index()
    
    total = gg.groupby("distance")["slot"].sum().to_dict()
    
    gg["slot"] = gg.apply(lambda x: x["slot"] / total[x["distance"]] * 100, axis=1)
    
    gg["per"] = gg["slot"]/ gg["slot"].sum() * 100
    
    #gg["distance"] = gg["distance"] -1


    fig = go.Figure()

    for ix, i in enumerate(gg["re-included"].unique()):
        _gg = gg[gg["re-included"] == i]
        fig.add_trace(go.Bar(
            x= _gg["distance"],
            y= _gg["slot"],
            marker_color=COLORS[ix],
            name=f'{i}',
        ))
    fig.update_layout(
            title_text=f'Attestation Vote Inclusion Delay - Percentage over Slot Delay <span style="font-size: 14px;">({slot_to_day(startslot)} - {slot_to_day(endslot)})</span>',
            legend=dict(x=0.82, y=0.9),
            barmode='stack',
            legend_traceorder="reversed",
            xaxis=dict(
                tickmode='linear',
                showgrid=True,
                gridcolor='lightgrey',
                title="delay in slots",
                range=[-0.75, 62]
            ),
            yaxis=dict(
                title="% attestations",
                showgrid=True,
                gridcolor='lightgrey',
                #type="log",
                #range=[0,1]

            ),
            height=550,
            width=1200,
            font=dict(
                family="Ubuntu Mono",
                size=18,
                color="black"
            ),
            plot_bgcolor = "#FFFFFF"

        )
    #fig.add_annotation(
    #    text=f'0 slot inclusion: {gg[gg["distance"] == 0]["per"].sum():,.2f}%',
    #    xref="paper",
    #    yref="paper",
    #    x=1,
    #    y=1,
    #    showarrow=False,
    #    font=dict(
    #        family="Ubuntu Mono",
    #        size=18,
    #        color="black"
    #    )
    #)
    return fig
if WHAT == "head":
    fig = correct__delay_reinclusion_per(df)
    fig.write_image(f"{IMAGE}/correct_{WHAT}_delay_reinclusion_per.png")
    if SHOW:
        fig.show()
    print("done")

In [ ]:
def correct_delay_clients(df):
    startslot = 9271808
    endslot = max(df.slot)
    gg = pd.DataFrame()
    l = len(df.epoch.unique())
    for ix, i in enumerate(sorted(df.epoch.unique())):
        print(f"{ix}/{l}", end="\r")
        #if ix > 3:
        #    break
        g = pd.read_parquet(f"{DATA}/correct_{WHAT}/{i}.parquet").drop_duplicates()
        g = g[g["validators"] != 999999999999999]
        g = pd.merge(g, clients_final, how="left", left_on="validators", right_on="validator_id")
        g["cl_client"] = g["cl_client"].fillna("Unknown")
        g = g.groupby(["cl_client", "distance"])["slot"].count().reset_index()
        gg = pd.concat([gg, g], ignore_index=True)

    gg = gg.groupby(["cl_client", "distance"])["slot"].sum().reset_index()
    gg["per"] = gg["slot"]/ gg["slot"].sum() * 100
    
    #gg["distance"] = gg["distance"] -1


    fig = go.Figure()

    for ix, i in enumerate(gg.cl_client.unique()):
        _gg = gg[gg["cl_client"] == i]
        fig.add_trace(go.Bar(
            x= _gg[_gg["distance"] != 1]["distance"],
            y= _gg[_gg["distance"] != 1]["per"],
            marker_color=COLORS[ix],
            name=f'{i}',
        ))
    fig.update_layout(
            title_text=f'Attestation Inclusion Delay <span style="font-size: 14px;">({slot_to_day(startslot)} - {slot_to_day(endslot)})</span>',
            legend=dict(x=0.87, y=0.9),
            barmode='stack',
            legend_traceorder="reversed",
            xaxis=dict(
                tickmode='linear',
                showgrid=True,
                gridcolor='lightgrey',
                title="delay in slots",
                range=[0.5, 63]
            ),
            yaxis=dict(
                title="% of attestations",
                showgrid=True,
                gridcolor='lightgrey',
                #type="log",
                #range=[0,1]

            ),
            height=550,
            width=1200,
            font=dict(
                family="Ubuntu Mono",
                size=18,
                color="black"
            ),
            plot_bgcolor = "#FFFFFF"

        )
    fig.add_annotation(
        text=f'1 slot inclusion: {gg[gg["distance"] == 0]["per"].sum():,.2f}%',
        xref="paper",
        yref="paper",
        x=1,
        y=1,
        showarrow=False,
        font=dict(
            family="Ubuntu Mono",
            size=18,
            color="black"
        )
    )
    return fig
if WHAT == "head":
    fig = correct_delay_clients(df)
    fig.write_image(f"{IMAGE}/correct_{WHAT}_delay_clients.png")
    if SHOW:
        fig.show()
    print("done")

In [ ]:
def correct_delay(df):
    startslot = 9271808
    endslot = max(df.slot)
    gg = pd.DataFrame()
    for i in sorted(df.epoch.unique()):
        print(i)
        g = pd.read_parquet(f"{DATA}/correct_{WHAT}/{i}.parquet").drop_duplicates()
        g = g[g["validators"] != 999999999999999]
        g = g.groupby("distance")["validators"].count().reset_index()
        gg = pd.concat([gg, g], ignore_index=True)

    gg = gg.groupby("distance")["validators"].sum().reset_index()
    gg["per"] = gg["validators"]/ gg["validators"].sum() * 100
    
    #gg["distance"] = gg["distance"] -1


    fig = go.Figure()

    fig.add_trace(go.Bar(
        x= gg[gg["distance"] != 1]["distance"],
        y= gg[gg["distance"] != 1]["per"],
        marker_color=COLORS[0],
        name='Performance Delta comparing `Actual` with `Expected`',
    ))
    fig.update_layout(
            title_text=f'{WHAT[0].upper() + WHAT[1:]} Vote Inclusion Delay <span style="font-size: 14px;">({slot_to_day(startslot)} - {slot_to_day(endslot)})</span>',
            legend=dict(x=0, y=1),
            barmode='group',
            legend_traceorder="reversed",
            xaxis=dict(
                tickmode='linear',
                showgrid=True,
                gridcolor='lightgrey',
                title="Delay in Slots",
                range=[1.5, 63]
            ),
            yaxis=dict(
                title="% of validators",
                showgrid=True,
                gridcolor='lightgrey',
                #type="log"

            ),
            height=550,
            width=1200,
            font=dict(
                family="Ubuntu Mono",
                size=18,
                color="black"
            ),
            plot_bgcolor = "#FFFFFF"

        )
    fig.add_annotation(
        text=f'0 slot inclusion: {gg[gg["distance"] == 0].iloc[0]["per"]:,.2f}%',
        xref="paper",
        yref="paper",
        x=0.98,
        y=1,
        showarrow=False,
        font=dict(
            family="Ubuntu Mono",
            size=18,
            color="black"
        )
    )
    return fig
#fig = correct_delay(df)
#fig.write_image(f"{IMAGE}/correct_{WHAT}_delay.png")
#if SHOW:
#    fig.show()
#print("done")

In [ ]:
#def get_correct_attestations(epoch, what="head"):
#    _df = pd.read_parquet(f"{DATA}/correct_{what}/{epoch}.parquet").reset_index(drop=True)
#    if not _df.slot.nunique() == 32:
#        return False
#    return _df
#
#def query_attestations(slot):
#    if slot > END_SLOT:
#        raise
#    _slot0 = slot // 32 * 32
#    _slot1 = _slot0 + 32
#    print(f"query_attestations(); slot at {slot}")
#    _query_attestations = f"""
#    SELECT DISTINCT
#        block_slot, slot, validators, source_root, target_root, beacon_block_root
#    FROM default.canonical_beacon_elaborated_attestation
#    WHERE
#        event_date_time > NOW() - INTERVAL '100 day'
#        AND meta_network_name = 'mainnet'
#        AND slot >= {_slot0}
#        AND slot < {_slot1}
#    """
#    return _query_attestations

In [ ]:
#qa = query_attestations((END_SLOT-7200)//32*32)
#res = request_query(qa, ["block_slot", "slot", "validators", "source_root", "target_root", "beacon_block_root"])
#res["validators"] = res["validators"].apply(lambda x: eval(x))
#for attestation_slot in res.slot.unique():
#    print(attestation_slot)
#    break

In [ ]:
#def hex_to_rgba(hex_color, alpha=1.0):
#    hex_color = hex_color.lstrip('#')
#    return f'rgba({int(hex_color[0:2], 16)}, {int(hex_color[2:4], 16)}, {int(hex_color[4:6], 16)}, {alpha})'
#
#def weak_timing_chart(_df):
#    #df = df.sample(1_000_000)
#    df = _df.copy()
#    gg = df.groupby("slot")["validator"].nunique().reset_index().sort_values("validator")
#    gg["per_voting_wrong"] = gg["validator"]/32175*100
#    block_timing = load_timing_data(df, overwrite=OVERWRITE, rounding=3)
#    gg = pd.merge(gg, block_timing, left_on="slot", right_on="slot")
#    gg = gg[gg["time_in_slot"] > 0]
#    gg = gg[["slot", "per_voting_wrong", "time_in_slot"]]
#    mev = pandas_gbq.read_gbq(f"""
#        SELECT distinct slot, builder FROM `ethereum-data-nero.eth.mevboost_db` WHERE slot >= {df.slot.min()}
#        order by slot
#    """)
#    gg = pd.merge(gg, mev, left_on="slot", right_on="slot")
#    largest = gg.groupby("builder")["slot"].count().reset_index().sort_values("slot", ascending=False)["builder"].tolist()[0:10]
#
#    
#    fig = go.Figure()
#
#    fontfamily = "Ubuntu Mono"
#    builder_colors= {i: COLORS3[ix] for ix, i in enumerate(gg.builder.unique())}
#
#    for builder in largest[:3]:
#        color = builder_colors[builder]
#        builder_df = gg[gg['builder'] == builder].sample(21000)
#        fig.add_trace(go.Scatter(
#            x=builder_df['time_in_slot'],
#            y=builder_df['per_voting_wrong'],
#            mode='markers',
#            name=builder,
#            marker=dict(size=4, color=hex_to_rgba(color, 0.3)),
#        ))
#
#    fig.update_layout(
#        title={
#            'text': f'Failed {WHAT[0].upper()+WHAT[1:]} Votes vs. Block Size <span style="font-size: 16px;">({slot_to_day(_df.slot.min())} - {slot_to_day(_df.slot.max())})</span>',
#            'font': dict(family=fontfamily, size=24, color='#333')
#        },
#        xaxis_title="failed votes per slot",
#        yaxis_title="compressed block size (incl. blobs)",
#        font=dict(
#            family=fontfamily,
#            size=FONTSIZE,
#            color="black"
#        ),
#        legend=dict(
#            font=dict(size=16),
#            itemsizing='constant',
#            tracegroupgap=5,
#        ),
#        paper_bgcolor='white',
#        plot_bgcolor='white',
#        xaxis=dict(
#            showgrid=True,
#            gridwidth=1,
#            gridcolor='LightGray',
#            zerolinecolor='gray',
#        ),
#        yaxis=dict(
#            showgrid=True,
#            gridwidth=1,
#            gridcolor='LightGray',
#            zerolinecolor='gray',
#           # type="log"
#        ),
#        showlegend=True,
#        height=550,
#        width=1200,
#    )
#    return fig
#
#if WHAT != "source":
#    fig = weak_timing_chart(df)
#    fig.write_image(f"{IMAGE}/{WHAT}_weak_timing_chart.png")
#    #if SHOW:
#        #fig.show()
#    print("done")

In [ ]:
#fig, ax = plt.subplots(figsize=(12, 20))
#
#joyplot(
#    data=gg[gg["builder"].isin(largest[:3])],
#    by='builder',
#    overlap=0.5,
#    column='time_in_slot',
#    figsize=(12, 20),
#    colormap={i: COLORS3[ix] for ix, i in enumerate(gg.builder.unique())},
#    #x_range=[0, 8],
#    alpha=0.5,
#    ax=ax
#)
#
#plt.xlabel('seconds in slot')
#plt.grid(True, axis='x')
#plt.text(-0.95, 1.0, f'Head Vote Seen Timing over Seconds In Slot (epoch {gg.slot.min() //32:,} - {gg.slot.max() // 32:,})', fontsize=20, ha='left', va='top', fontweight='bold', fontfamily='Ubuntu Mono')
#
#plt.savefig(f"{IMAGE}/{WHAT}_weak_timing_chart.png", format='png', transparent=True)
#plt.close()

In [ ]:
#def weak_timing_chart(_df):
#    df = _df.copy()
#    
#    fig = go.Figure()
#
#    fontfamily = "Ubuntu Mono"
#    builder_colors = {i: COLORS3[ix] for ix, i in enumerate(gg.builder.unique())}
#
#    # Aggregate data from all builders
#    aggregated_time_in_slot = []
#    aggregated_per_voting_wrong = []
#
#    for builder in largest[:1]:
#        builder_df = df[df['builder'] == builder]
#        aggregated_time_in_slot.extend(builder_df['time_in_slot'])
#        aggregated_per_voting_wrong.extend(builder_df['per_voting_wrong'])
#    
#    # Create a single 2D histogram
#    fig.add_trace(go.Histogram2d(
#        x=aggregated_time_in_slot,
#        y=aggregated_per_voting_wrong,
#        colorscale='Viridis',
#        nbinsx=10,  # Number of bins in x-direction
#        nbinsy=10,  # Number of bins in y-direction
#        colorbar=dict(title='Count')
#    ))
#
#    fig.update_layout(
#        title={
#            'text': f'Failed {WHAT[0].upper()+WHAT[1:]} Votes vs. Block Size <span style="font-size: 16px;">({slot_to_day(_df.slot.min())} - {slot_to_day(_df.slot.max())})</span>',
#            'font': dict(family=fontfamily, size=24, color='#333')
#        },
#        xaxis_title="failed votes per slot",
#        yaxis_title="compressed block size (incl. blobs)",
#        font=dict(
#            family=fontfamily,
#            size=FONTSIZE,
#            color="black"
#        ),
#        legend=dict(
#            font=dict(size=16),  # Increase the font size of the legend
#            itemsizing='constant',  # Ensures that the legend items are sized properly
#            tracegroupgap=5,  # Increase the space between legend items
#        ),
#        paper_bgcolor='white',
#        plot_bgcolor='white',
#        xaxis=dict(
#            showgrid=True,
#            gridwidth=1,
#            gridcolor='LightGray',
#            zerolinecolor='gray',
#        ),
#        yaxis=dict(
#            showgrid=True,
#            gridwidth=1,
#            gridcolor='LightGray',
#            zerolinecolor='gray',
#            # type="log"
#        ),
#        showlegend=True,
#        height=550,
#        width=1200,
#    )
#    return fig
#
#if WHAT != "source":
#    fig = weak_timing_chart(gg)
#    fig.write_image(f"{IMAGE}/{WHAT}_weak_timing_chart.png")
#    #if SHOW:
#        #fig.show()
#    print("done")

In [ ]:
print("start check")
slots = sorted(df.slot.unique(), reverse=True)
last = slots[0] +1
for i in slots:
    if last -1 != i:
        print(i)
        
    last = i
print("end check")
df = None
block_timing = None

In [ ]:
if not DO_PERFORMERS:
    raise

In [ ]:
ADD_MORE = False
ADD_MAX = 225*3
USE_MAX = 225*2*32

CORRECT_HEAD_TIMING = "correct_head_timing"

wt = sorted([int(i.split(".")[0]) for i in os.listdir(f"{DATA}/{CORRECT_HEAD_TIMING}")], reverse=True)

def get_time_in_curr_slot(ts, slot):
    return (ts - (1654824023000 + (slot-1-4e6)*12000) - 12000) / 1000 

def parse_datetime(event_time_str):
    return datetime.strptime(event_time_str, '%Y-%m-%d %H:%M:%S.%f').replace(tzinfo=pytz.UTC).timestamp()*1000

def get_time_in_curr_slot2(event_time_str, slot):
    event_time = datetime.strptime(event_time_str, '%Y-%m-%d %H:%M:%S.%f').timestamp()*1000
    return (event_time - (1654824023000 + (slot-1-4e6)*12000) - 12000) / 1000 

if "labels" not in globals():
    labels=None
    
def get_timing_dataset():
    global labels
    try:
        df = pd.read_parquet(f"{DATA}/{TMP}/correct_{WHAT}_timing.parquet").drop_duplicates()
        #df.drop("event_date_time", axis=1, inplace=True)
        
        #df["epoch"] = df["slot"] //32
        #print(df.epoch.nunique())
        df = df[df["slot"] >= (df.slot.max() - USE_MAX)//32*32]
        #print(df.epoch.nunique())
        #h = df.groupby("epoch")["slot"].nunique().reset_index()
        known_epochs = set(df["slot"].unique() // 32)
        #df.drop("epoch", axis=1, inplace=True)
        #known_epochs = set(df.slot // 32)
        print(f"{DATA}/{TMP}/correct_{WHAT}_timing.parquet loaded locally")
        print(f"{len(known_epochs)} epochs known")
        l = len(known_epochs)
        #print(f"storing dataset {DATA}/{TMP}/correct_{WHAT}_timing.parquet")
        #df.drop_duplicates(inplace=True)
        #df.to_parquet(f"{DATA}/{TMP}/correct_{WHAT}_timing.parquet", index=False)
    except Exception as e:
        print(e)
        df = pd.DataFrame()
        known_epochs = set()

    change=False
    try:
        if ADD_MORE:
            lastepoch = None
            for ix, epoch in enumerate(wt):
                if ix == 0:
                    lastepoch = epoch
                else:
                    if lastepoch - 1 != epoch:
                        print("exit")
                        raise
                    else:
                        lastepoch = epoch
                if epoch in known_epochs:
                    continue
                #if epoch > 286521:
                #    continue
                print(epoch, ix, ADD_MAX)
                if ix + 1 > ADD_MAX:
                    print("exit")
                    raise
                dfwt = pd.read_parquet(f"{DATA}/{CORRECT_HEAD_TIMING}/{epoch}.parquet").drop_duplicates()
                #dfwt.drop("event_date_time", axis=1, inplace=True)
                dfwt['event_date_time'] = dfwt['event_date_time'].apply(lambda x: parse_datetime(x))
                dfwt["seconds_in_slot"] = dfwt.apply(lambda x: get_time_in_curr_slot(x["event_date_time"], x["slot"]), axis=1)
                dfwt.drop("event_date_time", axis=1, inplace=True)
                print(f"adding {len(dfwt)} lines for epoch {epoch}")
                df = pd.concat([dfwt, df], ignore_index=True)
                change = True
        raise

    except Exception as e:
        print(e)

    finally:
        if change:
            print(f"storing dataset {DATA}/{TMP}/correct_{WHAT}_timing.parquet")
            df.drop_duplicates(inplace=True)
            df.to_parquet(f"{DATA}/{TMP}/correct_{WHAT}_timing.parquet", index=False)
            print(f"{len(set(df.slot // 32))} epochs in total")
        else:
            print("nothing changed")
        df = df[["slot", "attesting_validator_index", "seconds_in_slot"]]
        df.columns=["slot", "validator", "seconds_in_slot"]
        labels = get_labels(overwrite=OVERWRITE)
        totals = get_totals(df.slot.max()//32, nlargest=30)
        df, largest = merge_labels(df, labels, nlargest=30)
        print("-")
        jj = pd.DataFrame()
        for i in df.label.unique():
            print(i, end="\r")
            _df = df[df["label"] == i].copy()
            _df = _df.sample(n=min([15000, len(_df)]), random_state=42)
            jj = pd.concat([jj,_df], ignore_index=True)  
        return df, largest, totals, jj
    
if WHAT == "head":
    gg, largest, totals, jj = get_timing_dataset()
    print(f"{gg.slot.nunique() // 32} epochs in dataset")

In [ ]:
def create_best_performer_list():
    DAYS = 3
    df = read_df()
    df = df[df["epoch"] > df.epoch.max() - DAYS*225]
    a1 = set(check_duties(df.epoch.min())["validator"])
    a2 = set(check_duties(df.epoch.max())["validator"])
    a0 = a1.intersection(a2)
    al = sorted(a0 - set(df.validator.tolist()))
    with open(f"{DATA}/{TMP}/high_performers.txt", "w") as file:
        file.write(str(al))
    print("best list created")
    
if WHAT == "head":
    if CREATE_BEST_LIST:
        create_best_performer_list()

## Best Performers

In [ ]:
if WHAT == "head":
    with open(f"{DATA}/{TMP}/high_performers.txt", "r") as file:
        al = eval(file.read())

    best = gg.groupby(["validator", "label"])["slot"].nunique().reset_index().sort_values("slot", ascending=False)

    #best = best[best["slot"] == best["slot"].max()].reset_index(drop=True)

    best = best[best["validator"].isin(al)]
    zeros = set(gg.label.unique())- set(best.label.unique())
    for i in zeros:
        best.loc[len(best), ("label", "validator", "slot")] = (i, 999999999999999, 0)

    best = best.groupby("label")["validator"].nunique().reset_index().sort_values("validator", ascending=False)
    best = pd.merge(best, totals, how="left", left_on="label", right_on="label")
    best.dropna(inplace=True)
    best["best_per"] = best["validator_x"] / best["validator_y"] * 100

    best.loc[best["label"].isin(zeros), "validator_x"] = 0
    best.loc[best["label"].isin(zeros), "best_per"] = 0
    _labels = set(best.label)
    _largest = [i for i in largest if i in _labels]
    best.set_index("label", inplace=True)
    best = best.loc[_largest]
    best.reset_index(inplace=True)
    best

In [ ]:
if WHAT == "head":
    table = best[["label", "validator_x", "validator_y", "best_per"]]
    table.columns = ["Entity", "Nr. of<br>high-performing<br>Validators", "Total<br>Validators", "% of<br>high-performing<br>Validators"]
    table.set_index("Entity", inplace=True)

    # Apply rounding, type conversion, and thousand separators
    for i in table.columns:
        if "%" in i:
            table[i] = table[i].apply(lambda x: round(x, 2))
        else:
            table[i] = table[i].apply(lambda x: "{:,}".format(x))

    # Function to apply color formatting
    def color_high_low(val):
        color = ''
        if isinstance(val, (int, float)):
            if val > table["% of<br>high-performing<br>Validators"].mean():
                color = 'background-color: lightgreen'
            elif val < table["% of<br>high-performing<br>Validators"].mean():
                color = 'background-color: lightcoral'
        return color

    # Apply the function to the last column using Styler.applymap
    styled_table = table.style.applymap(color_high_low, subset=["% of<br>high-performing<br>Validators"])

    epoch_min = gg['slot'].min() // 32
    epoch_max = gg['slot'].max() // 32
    subtitle = f'''
    ____________________________________________________<br>
    <span style="font-size: 14px;font-family:Ubuntu mono">epoch {epoch_min:,} - {epoch_max:,} ({(epoch_max-epoch_min)} epochs)<br>
    high performer = validator with correct head votes for {3} days
    </span>'''

    # Use tabulate to print the table in markdown format with headers having line breaks
    print(tabulate(table, headers='keys', tablefmt='pipe'))

    # Convert the styled table to HTML and add the subtitle
    styled_table_html = styled_table.set_table_styles([
        {'selector': 'table', 'props': [('width', '100%'), ('font-size', '14pt')]},
        {'selector': 'th, td', 'props': [('padding-right', '10px'), ('padding-left', '10px'), ('text-align', 'right'), ('font-size', '10pt')]},
        {'selector': 'th', 'props': [('word-wrap', 'break-word'), ('white-space', 'normal'), ('font-size', '10pt')]}
    ]).format({
        "% of<br>high-performing<br>Validators": "{:.2f}"
    }).to_html() + subtitle


    # Optionally save the styled table to an HTML file
    with open(f'{DATA}/{TMP}/performer_table.html', 'w') as f:
        f.write(styled_table_html)

    options = {
        'width': '650',  # Set the desired width
        'height': '770'  # Set the desired height
    }

    # Convert the HTML file to PNG using imgkit with specified options
    imgkit.from_file(f'{DATA}/{TMP}/performer_table.html', f'{IMAGE}/performer_table.jpeg', options=options)

In [ ]:
def high_performer_share(best):
    _best = best[["label", "best_per"]].copy()
    fig = go.Figure()
    start_color = '#377eb8'
    end_color = '#8cb6fa'

    # Generate a gradient of 30 colors between the start and end colors
    cmap = mcolors.LinearSegmentedColormap.from_list("custom_gradient", [start_color, end_color])
    gradient = [mcolors.rgb2hex(cmap(i / 30)) for i in range(31)]

    # If there are more than 30 bars, repeat the gradient
    num_bars = len(_best)
    colors = (gradient * ((num_bars // 31) + 1))[:num_bars]

    fig.add_trace(go.Bar(
        x=_best["label"],
        y=_best["best_per"],
        marker=dict(color=colors),
        name=None
    ))

    fig.update_layout(
        title_text=f'High Performers over Node Operators <span style="font-size: 16px;">(epoch {gg.slot.min() //32:,} - {gg.slot.max() // 32:,})</span>',
        legend=dict(x=0, y=1),
        barmode='group',
        legend_traceorder="reversed",
        xaxis=dict(
            tickmode='linear',
            showgrid=True,
            gridcolor='lightgrey',
            title=None,
        ),
        yaxis=dict(
            title="% high performers of total",
            showgrid=True,
            gridcolor='lightgrey',
        ),
        height=550,
        width=1200,
        font=dict(
            family="Ubuntu Mono",
            size=16,
            color="black"
        ),
        plot_bgcolor="#FFFFFF"
    )
    return fig
if WHAT == "head":
    fig = high_performer_share(best)
    fig.write_image(f"{IMAGE}/topperformer_percentage.png")
    fig.show()

In [ ]:
def high_performer_vs_rest_timing(gg):
    tt = gg.groupby(["validator"])["seconds_in_slot"].median().reset_index().sort_values("seconds_in_slot", ascending=False)

    rr = gg.groupby(["validator", "label"])["slot"].nunique().reset_index().sort_values("slot", ascending=False)
    rr = rr[rr["validator"].isin(al)].reset_index(drop=True)
    #rr = rr[rr["slot"] == rr["slot"].max()].reset_index(drop=True)
    rr = pd.merge(rr, tt, how="left", left_on="validator", right_on="validator")

    #if not "jj" in globals():
    #    jj = pd.DataFrame()
    #    for i in gg.label.unique():
    #        print(i, end="\r")
    #        _df = gg[gg["label"] == i].copy()
    #        _df = _df.sample(n=min([1000, len(_df)]), random_state=42)
    #        jj = pd.concat([jj,_df], ignore_index=True)  

    def calculate_kde(data, x_range):
        kde = gaussian_kde(data)
        x = np.linspace(x_range[0], x_range[1], 1000)
        y = kde(x)
        return x, y

    x_jj, y_jj = calculate_kde(jj['seconds_in_slot'], (0, 10))
    x_rr, y_rr = calculate_kde(rr['seconds_in_slot'], (0, 10))

    fig = go.Figure()

    # Add KDE plot for the first dataset
    fig.add_trace(go.Scatter(
        x=x_jj,
        y=y_jj,
        fill='tozeroy',
        mode='lines',
        name='Rest',
        line=dict(color=COLORS3[0]),
        opacity=0.5
    ))

    # Add KDE plot for the second dataset
    fig.add_trace(go.Scatter(
        x=x_rr,
        y=y_rr,
        fill='tozeroy',
        mode='lines',
        name=f'High Performer (n={len(rr)})',
        line=dict(color=COLORS3[1]),
        opacity=0.5
    ))


    fig.update_layout(
            title=f'Head Vote Timing (first seen) - High Performer vs. Rest <span style="font-size: 16px;">(epoch {gg.slot.min() //32:,} - {gg.slot.max() // 32:,})</span>',
            yaxis_title='density',
            xaxis_title='seconds in slot',
            barmode='stack',
            legend_traceorder="normal",
            xaxis=dict(
                tickmode='linear',
                showgrid=True,
                gridcolor='lightgrey',
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgrey',
            ),
            height=IMAGE_HEIGHT,
            width=1200,
            font=dict(
                family="Ubuntu Mono",
                size=FONTSIZE,
                color="black"
            ),
            legend=dict(
                x=1,
                y=1,
                xanchor='right',
                yanchor='top'
            ),
            plot_bgcolor = "#FFFFFF"
            #yaxis = dict(
            #    type="log"
            #)
        )

    # Remove border (Plotly does not have a direct way to remove axis borders, so we set them to be invisible)
    fig.update_xaxes(showline=False)
    fig.update_yaxes(showline=False)
    return fig

if WHAT == "head":
    fig = high_performer_vs_rest_timing(gg)
    fig.write_image(f"{IMAGE}/high_performer_vs_rest_timing.png")
    if SHOW:
        fig.show()

In [ ]:
def save_joyplot(_jj, colormap, filename, _largest):
    rr = _jj.copy()
    
    #rr.set_index("label", inplace=True)
    #rr = rr.loc[_largest]
    #rr.reset_index(inplace=True)
    rr['label'] = pd.Categorical(rr['label'], categories=_largest, ordered=True)
    print(len(rr))
    #rr = rr.sample(n=min([9790, len(rr)]), random_state=42)

    fig, ax = plt.subplots(figsize=(12, 20))
    
    joyplot(
        data=rr,
        by='label',
        overlap=0.5,
        column='seconds_in_slot',
        figsize=(12, 20),
        colormap=colormap,
        x_range=[0, 8],
        alpha=0.5,
        ax=ax
    )
    
    plt.xlabel('seconds in slot')
    plt.grid(True, axis='x')
    plt.text(-0.95, 1.0, f'Head Vote Seen Timing over Seconds In Slot (epoch {gg.slot.min() //32:,} - {gg.slot.max() // 32:,})', fontsize=20, ha='left', va='top', fontweight='bold', fontfamily='Ubuntu Mono')

    plt.savefig(filename, format='png', transparent=True)
    plt.close()

def timing_head_votes_ridgelines(gg, al):
    rr = gg.copy()
    rr = rr[rr["validator"].isin(al)].reset_index(drop=True)
    
    # Ensure that 'seconds_in_slot' is numeric
    rr['seconds_in_slot'] = pd.to_numeric(rr['seconds_in_slot'], errors='coerce')

    COLORS3 = [
        '#377eb8', '#ff7f00', '#4daf4a', '#984ea3', '#f781bf', '#a65628',
        '#e41a1c', '#a6cee3', '#999999', '#fdbf6f', '#b2df8a', '#fb9a99',
        '#cab2d6', '#1f78b4', '#33a02c', '#ff7f7f', '#8dd3c7', '#ffffb3',
        '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69', '#fccde5',
        '#d9d9d9', '#bc80bd', '#ccebc5', '#ffed6f',
        '#6a3d9a', '#ffcc00', '#b15928', '#1f78b4', '#e31a1c', '#33a02c',
        '#fb9a99', '#e6ab02', '#a6761d', '#666666'
    ]

    colors1 = [COLORS3[0], '#a3d6ff']
    cmap1 = LinearSegmentedColormap.from_list('transition1', colors1, N=30)

    colors2 = ['#ff7f00', '#ffd8b1']
    cmap2 = LinearSegmentedColormap.from_list('transition2', colors2, N=30)

    # Sort the DataFrame by 'label' while preserving the existing order
    rr['label'] = pd.Categorical(rr['label'], categories=rr['label'].unique(), ordered=True)
    rr_sorted = rr.sort_values('label')
    
    font_path = '/usr/share/fonts/truetype/dejavu/UbuntuMono-R.ttf'  # Update this with the actual path to the font file
    font_manager.fontManager.addfont(font_path)
    plt.rcParams['font.family'] = 'Ubuntu Mono'
    plt.rcParams['font.size'] = 14
    plt.rcParams['axes.titlesize'] = 20
    plt.rcParams['axes.labelsize'] = 16
    plt.rcParams['xtick.labelsize'] = 14
    plt.rcParams['ytick.labelsize'] = 14
    labels = set(jj.label)
    _largest = [i for i in largest if i in labels]
    save_joyplot(jj, cmap1, f"{DATA}/{TMP}/joyplot1.png", _largest)
    save_joyplot(rr, cmap2, f"{DATA}/{TMP}/joyplot2.png", _largest)


def create_legend(output_path):
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.axis('off')

    rest_patch = mpatches.Patch(color='#377eb8', label='Rest', alpha=0.5)
    high_perf_patch = mpatches.Patch(color='#ff7f00', label='High Performers', alpha=0.5)

    ax.legend(handles=[high_perf_patch, rest_patch], loc='center', fontsize=24, frameon=True, framealpha=1)
    plt.savefig(output_path, format='png', transparent=True)
    plt.close()

def overlay_images(image1_path, image2_path, output_path):
    image1 = Image.open(image1_path).convert("RGBA")
    image2 = Image.open(image2_path).convert("RGBA")

    combined = Image.alpha_composite(image1, image2)
    combined.save(output_path)

def overlay_legend(combined_image_path, legend_path, final_output_path):
    combined_image = Image.open(combined_image_path).convert("RGBA")
    legend = Image.open(legend_path).convert("RGBA")

    # Resize legend if needed
    legend = legend.resize((int(combined_image.width * 0.3), int(combined_image.height * 0.1)))
    
    legend_position = (combined_image.width - legend.width - 50, 25)

    # Paste the legend on the combined image at a specified position
    combined_image.paste(legend, (860,-20), legend)
    combined_image.save(final_output_path)
if WHAT == "head":
    timing_head_votes_ridgelines(jj, al)
    # Ensure the legend image is created and saved correctly
    create_legend(f"{DATA}/{TMP}/legend.png")

    # Overlay the saved joyplot images
    overlay_images(f"{DATA}/{TMP}/joyplot1.png", f"{DATA}/{TMP}/joyplot2.png", f"{DATA}/{TMP}/combined_joyplot.png")

    # Overlay the legend on the combined image
    overlay_legend(f"{DATA}/{TMP}/combined_joyplot.png", f"{DATA}/{TMP}/legend.png", f"{IMAGE}/ridgeline_timing.png")

    # Display the final combined image with legend
    final_combined_image = Image.open(f"{IMAGE}/ridgeline_timing.png")
    white_background = Image.new("RGBA", final_combined_image.size, "WHITE")

    # Paste the original image onto the white background
    white_background.paste(final_combined_image, (0, 0), final_combined_image)

    # Convert to RGB to remove the alpha channel
    final_image_with_white_bg = white_background.convert("RGB")

    # Save the final image
    final_image_with_white_bg.save(f"{IMAGE}/ridgeline_timing_with_white_bg.png")

    # Display the final image
    final_image_with_white_bg.show()

In [ ]:
#clients_final = get_clients(overwrite=OVERWRITE)

clients_final = get_clients(overwrite=False)
if WHAT == "head":
    
    _gg = pd.merge(gg, clients_final, how="left", left_on="validator", right_on="validator_id")
    _jj = pd.merge(jj, clients_final, how="left", left_on="validator", right_on="validator_id")
    _jj.fillna("Unknown", inplace=True)
    _gg.fillna("Unknown", inplace=True)
    _jj = _jj[_jj["seconds_in_slot"] <= 12]

    cl_client_counts = _gg.groupby("cl_client")["slot"].count().to_dict()
    _largest = ["Lighthouse", "Prysm", "Teku", "Nimbus", "Lodestar", "Unknown"]

def save_joyplot(_jj, colormap, filename, _largest):
    rr = _jj.copy()
    
    #rr.set_index("label", inplace=True)
    #rr = rr.loc[_largest]
    #rr.reset_index(inplace=True)
    rr['cl_client'] = pd.Categorical(rr['cl_client'], categories=_largest, ordered=True)
    print(len(rr))
    #rr = rr.sample(n=min([9790, len(rr)]), random_state=42)

    fig, ax = plt.subplots(figsize=(12, 6))
    
    joyplot(
        data=rr,
        by='cl_client',
        overlap=0.7,
        column='seconds_in_slot',
        figsize=(12, 6),
        colormap=colormap,
        x_range=[0, 8],
        alpha=0.5,
        ax=ax
    )
    
    plt.xlabel('seconds in slot')
    plt.grid(True, axis='x')
    plt.text(-0.95, 1.0, f'Head Vote Seen Timing over Seconds In Slot (epoch {gg.slot.min() //32:,} - {gg.slot.max() // 32:,})', fontsize=20, ha='left', va='top', fontweight='bold', fontfamily='Ubuntu Mono')
    for idx, client in enumerate(_largest):
        count = cl_client_counts.get(client, 0)
        plt.text(7, 0.78 - 0.145*idx, f"n={count:,}", fontsize=14, ha='left', va='center')

    plt.savefig(filename, format='png', transparent=True)
    plt.close()

def timing_head_votes_clients():

    COLORS3 = [
        '#377eb8', '#ff7f00', '#4daf4a', '#984ea3', '#f781bf', '#a65628',
        '#e41a1c', '#a6cee3', '#999999', '#fdbf6f', '#b2df8a', '#fb9a99',
        '#cab2d6', '#1f78b4', '#33a02c', '#ff7f7f', '#8dd3c7', '#ffffb3',
        '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69', '#fccde5',
        '#d9d9d9', '#bc80bd', '#ccebc5', '#ffed6f',
        '#6a3d9a', '#ffcc00', '#b15928', '#1f78b4', '#e31a1c', '#33a02c',
        '#fb9a99', '#e6ab02', '#a6761d', '#666666'
    ]

    colors1 = [COLORS3[0], '#a3d6ff']
    cmap1 = LinearSegmentedColormap.from_list('transition1', colors1, N=30)

    # Sort the DataFrame by 'label' while preserving the existing order
    #_jj['label'] = pd.Categorical(_jj['label'], categories=_jj['label'].unique(), ordered=True)
    #_jj_sorted = _jj.sort_values('label')
    
    font_path = '/usr/share/fonts/truetype/dejavu/UbuntuMono-R.ttf'  # Update this with the actual path to the font file
    font_manager.fontManager.addfont(font_path)
    plt.rcParams['font.family'] = 'Ubuntu Mono'
    plt.rcParams['font.size'] = 14
    plt.rcParams['axes.titlesize'] = 20
    plt.rcParams['axes.labelsize'] = 16
    plt.rcParams['xtick.labelsize'] = 14
    plt.rcParams['ytick.labelsize'] = 14
    cl_client = set(_jj.cl_client)
    #_largest = _jj.cl_client.unique().tolist()
    #save_joyplot(jj, cmap1, f"{DATA}/{TMP}/joyplot1.png", _largest)
    save_joyplot(_jj, cmap1, f"{IMAGE}/head_timing_cl_clients.png", _largest)


def create_legend(output_path):
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.axis('off')

    rest_patch = mpatches.Patch(color='#377eb8', label='Rest', alpha=0.5)
    high_perf_patch = mpatches.Patch(color='#ff7f00', label='High Performers', alpha=0.5)

    ax.legend(handles=[high_perf_patch, rest_patch], loc='center', fontsize=24, frameon=True, framealpha=1)
    plt.savefig(output_path, format='png', transparent=True)
    plt.close()

def overlay_legend(combined_image_path, legend_path, final_output_path):
    combined_image = Image.open(combined_image_path).convert("RGBA")
    legend = Image.open(legend_path).convert("RGBA")

    # Resize legend if needed
    legend = legend.resize((int(combined_image.width * 0.3), int(combined_image.height * 0.1)))
    
    legend_position = (combined_image.width - legend.width - 50, 25)

    # Paste the legend on the combined image at a specified position
    combined_image.paste(legend, (860,-20), legend)
    combined_image.save(final_output_path)

if WHAT == "head":
    timing_head_votes_clients()
    create_legend(f"{DATA}/{TMP}/legend2.png")


    # Display the final combined image with legend
    final_image = Image.open(f"{IMAGE}/head_timing_cl_clients.png")
    final_image.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

def attestations_cdf(gg):

    data = gg.seconds_in_slot.tolist()[::100000]
    data.sort()

    data_array = np.array(data)

    cum_freq = np.cumsum(np.ones_like(data_array))
    cum_density = cum_freq / cum_freq[-1]

    cumulative_density = list(zip(data_array, cum_density))

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=data_array,
        y=cum_density,
        mode='lines',
        name='Cumulative Density',
        line=dict(color=COLORS[0], width=3),
    ))

    fig.update_layout(
            title=f'Cum. Distribution of Attestations First Seen Timing <span style="font-size: 16px;">(epoch {gg.slot.min() //32:,} - {gg.slot.max() // 32:,})</span>',
            yaxis_title='cumulative probability',
            xaxis_title='seconds in slot',
            barmode='stack',
            legend_traceorder="normal",
            xaxis=dict(
                tickmode='linear',
                showgrid=True,
                gridcolor='lightgrey',
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgrey',
            ),
            height=IMAGE_HEIGHT,
            width=1200,
            font=dict(
                family="Ubuntu Mono",
                size=FONTSIZE,
                color="black"
            ),
            legend=dict(
                x=1,
                y=1,
                xanchor='right',
                yanchor='top'
            ),
            plot_bgcolor = "#FFFFFF"
            #yaxis = dict(
            #    type="log"
            #)
        )

    # Remove border (Plotly does not have a direct way to remove axis borders, so we set them to be invisible)
    #fig.update_xaxes(showline=False)
    #fig.update_yaxes(showline=False)
    return fig

if WHAT == "head":
    fig = attestations_cdf(gg)
    fig.write_image(f"{IMAGE}/attestations_cdf.png")
    if SHOW:
        fig.show()
